In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Getting news data
import os
import time
import requests
import pandas as pd
from datetime import datetime, timedelta, timezone

# ================= CONFIGURATION ================= #
API_KEY = "MXZ437NRYQRY9FDY"   # Your Alpha Vantage Premium API key
OUTPUT_DIR = "news_data"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Example top 50 tickers (replace with your own list)
TICKERS = [
    "AAPL","MSFT","NVDA","GOOG","AMZN","META","TSLA","BRK.B","UNH","JNJ",
    "V","XOM","JPM","PG","MA","HD","CVX","LLY","ABBV","MRK",
    "PEP","KO","AVGO","COST","PFE","BAC","TMO","ADBE","DIS","CSCO",
    "WMT","NFLX","ABT","ACN","INTC","DHR","MCD","NKE","TXN","NEE",
    "VZ","LIN","PM","ORCL","UPS","BMY","HON","RTX","AMGN","IBM"
]

START_AT_STR = "2010-01-01 00:00:00Z"   # Start date (UTC)
THRESH = 0.25                           # Relevance threshold
LIMIT  = 1000                           # Max results per request
SORT   = "LATEST"                       # Sort by time descending
MIN_INTERVAL_S = 0.9                    # Sleep interval (Premium = 75 req/min)
BASE_URL = "https://www.alphavantage.co/query"
SESSION  = requests.Session()

# ================= AUTHORITY LEVEL MAPPING ================= #
TIER1 = {
    "Reuters","Associated Press","Wall Street Journal","New York Times",
    "MarketWatch","CNBC","Barrons"
}
TIER2 = {
    "Zacks Commentary","Forbes","Business Insider",
    "Investors Business Daily","The Street","The Financial Express"
}
TIER3 = {
    "Benzinga","Motley Fool","Money Morning","StockMarket.com",
    "Stocknews.com","Canada Newswire","PR Newswire","GlobeNewswire"
}

def get_authority_level(source: str) -> int:
    """Return authority level for each news source."""
    if not source:
        return 0
    s = source.strip()
    if s in TIER1:
        return 3
    if s in TIER2:
        return 2
    if s in TIER3:
        return 1
    return 1

# ================= HELPER FUNCTIONS ================= #
def parse_start(s: str) -> datetime:
    """Parse flexible start date string into a UTC datetime object."""
    s = s.strip()
    fmts = [
        "%Y-%m-%d %H:%M:%SZ","%Y-%m-%d %H:%M:%S",
        "%Y-%m-%d","%Y%m%dT%H%M%S","%Y%m%dT%H%M",
    ]
    for f in fmts:
        try:
            dt = datetime.strptime(s, f)
            if dt.tzinfo is None:
                dt = dt.replace(tzinfo=timezone.utc)
            return dt.astimezone(timezone.utc)
        except Exception:
            pass
    raise ValueError(f"Cannot parse START_AT_STR: {s}")

def fmt_av(dt: datetime) -> str:
    """Format datetime into Alpha Vantage compatible string (YYYYMMDDTHHMM)."""
    return dt.astimezone(timezone.utc).strftime("%Y%m%dT%H%M")

def parse_feed_time(tp: str):
    """Parse 'time_published' field into datetime."""
    for f in ("%Y%m%dT%H%M%S", "%Y%m%dT%H%M"):
        try:
            return datetime.strptime(tp, f).replace(tzinfo=timezone.utc)
        except Exception:
            continue
    return None

def safe_sleep(sec: float):
    """Rate limiting between API calls."""
    if sec and sec > 0:
        time.sleep(sec)

def call_av(params: dict, max_retries: int = 4, base_backoff: float = 1.5):
    """Make a request to Alpha Vantage API with exponential backoff."""
    tries = 0
    while True:
        tries += 1
        r = SESSION.get(BASE_URL, params=params, timeout=30)
        r.raise_for_status()
        j = r.json()

        # Normal response
        if isinstance(j.get("feed"), list):
            return j["feed"], None, j

        # Handle rate limiting or errors
        diag = j.get("Note") or j.get("Information") or j.get("Error Message")
        if diag and any(k in diag.lower() for k in ["frequency","limit","call","per minute","per day"]):
            if tries <= max_retries:
                wait_s = base_backoff ** tries
                print(f"[AV NOTE] {diag}\n-> Retrying after {wait_s:.1f}s (try {tries}/{max_retries})")
                time.sleep(wait_s)
                continue
        return [], diag or "Unknown reason", j

def fetch_chunk(ticker: str, t_from: datetime, t_to: datetime | None, limit: int = LIMIT):
    """Fetch a single batch of news articles for a given ticker and time range."""
    params = {
        "function": "NEWS_SENTIMENT",
        "tickers": ticker,
        "apikey": API_KEY,
        "limit": limit,
        "time_from": fmt_av(t_from),
        "sort": SORT,
    }
    if t_to:
        params["time_to"] = fmt_av(t_to)
    feed, diag, raw = call_av(params)
    safe_sleep(MIN_INTERVAL_S)
    return feed, diag, params, raw

# ================= CORE FUNCTION ================= #
def fetch_all_news_for_ticker(ticker: str, start_at: datetime, end_at: datetime):
    """Fetch all available news for a given ticker until reaching the start date."""
    seen_urls, all_rows = set(), []
    cursor_to = end_at
    batch_idx = 0
    max_loops = 20000

    while batch_idx < max_loops:
        batch_idx += 1
        feed, diag, params_used, raw_json = fetch_chunk(ticker, start_at, cursor_to, limit=LIMIT)

        # If feed is empty on the first try, test without 'time_to'
        if not feed:
            if batch_idx == 1:
                probe_params = {
                    "function": "NEWS_SENTIMENT",
                    "tickers": ticker,
                    "apikey": API_KEY,
                    "limit": min(50, LIMIT),
                    "time_from": fmt_av(start_at),
                    "sort": SORT,
                }
                probe_feed, probe_diag, _ = call_av(probe_params)
                safe_sleep(MIN_INTERVAL_S)
                if probe_feed:
                    print(f"[{ticker}][Batch {batch_idx}] feed=0; but probe found data → continuing.")
                    feed = probe_feed
                else:
                    print(f"[{ticker}][Batch {batch_idx}] feed=0; diag={diag} raw_keys={list(raw_json.keys())}")
                    break
            else:
                print(f"[{ticker}][Batch {batch_idx}] feed=0; diag={diag} raw_keys={list(raw_json.keys())}")
                break

        # Track oldest timestamp in this batch
        raw_oldest = None
        for it in feed:
            t = parse_feed_time(it.get("time_published",""))
            if t and (raw_oldest is None or t < raw_oldest):
                raw_oldest = t

        # Collect new filtered items
        oldest_new, new_count = None, 0
        for it in feed:
            url = it.get("url")
            if not url or url in seen_urls:
                continue
            t = parse_feed_time(it.get("time_published",""))
            if t is None or t < start_at or t > end_at:
                continue

            ts_list = it.get("ticker_sentiment") or []
            me = next((ts for ts in ts_list if ts.get("ticker") == ticker), None)
            if not me:
                continue
            rel = float(me.get("relevance_score") or 0.0)
            if rel <= THRESH:
                continue

            seen_urls.add(url)
            new_count += 1
            if oldest_new is None or t < oldest_new:
                oldest_new = t

            # Construct a clean row
            row = {
                "ticker": ticker,
                "relevance": rel,
                "source": it.get("source"),
                "authority_level": get_authority_level(it.get("source")),
                "time": t.strftime("%Y-%m-%d %H:%M:%S+0000"),
                "summary": it.get("summary") or "",
                "url": it.get("url") or "",
            }
            all_rows.append(row)

        print(
            f"[{ticker}][Batch {batch_idx}] raw={len(feed)}, new={new_count}; "
            f"cursor_to={cursor_to.isoformat()} "
            f"raw_oldest={raw_oldest.isoformat() if raw_oldest else 'N/A'} "
            f"oldest_new={oldest_new.isoformat() if oldest_new else 'N/A'}"
        )

        if raw_oldest and raw_oldest <= start_at:
            print(f"[{ticker}][Batch {batch_idx}] reached start date ({start_at.isoformat()}), stopping.")
            break

        next_cursor = (raw_oldest - timedelta(minutes=1)) if raw_oldest else (cursor_to - timedelta(hours=1))
        if not (next_cursor < cursor_to):
            next_cursor = cursor_to - timedelta(hours=1)
        cursor_to = next_cursor

    all_rows.sort(key=lambda x: x["time"], reverse=True)
    return all_rows

# ================= MAIN EXECUTION ================= #
def main():
    start_at = parse_start(START_AT_STR)
    now_utc  = datetime.now(timezone.utc)
    print(f"Fetching range: {start_at.isoformat()} ~ {now_utc.isoformat()} | Relevance > {THRESH}")

    for tk in TICKERS:
        try:
            rows = fetch_all_news_for_ticker(tk, start_at, now_utc)
            if not rows:
                print(f"[{tk}] No matching data.")
                continue

            df = pd.DataFrame(rows, columns=["ticker","relevance","source","authority_level","time","summary","url"])
            fname = f"{tk}_{start_at.strftime('%Y%m%d')}_{now_utc.strftime('%Y%m%d')}.xlsx"
            fpath = os.path.join(OUTPUT_DIR, fname)
            df.to_excel(fpath, index=False)
            print(f"[{tk}] Saved {len(df)} rows → {fpath}")
        except Exception as e:
            print(f"[{tk}] Failed: {e}")

if __name__ == "__main__":
    main()


Fetching range: 2010-01-01T00:00:00+00:00 ~ 2025-12-09T19:04:21.419519+00:00 | Relevance > 0.25
[AAPL][Batch 1] raw=1000, new=1000; cursor_to=2025-12-09T19:04:21.419519+00:00 raw_oldest=2025-10-18T22:29:00+00:00 oldest_new=2025-10-18T22:29:00+00:00
[AAPL][Batch 2] raw=1000, new=1000; cursor_to=2025-10-18T22:28:00+00:00 raw_oldest=2025-10-15T08:02:22+00:00 oldest_new=2025-10-15T08:02:22+00:00
[AAPL][Batch 3] raw=1000, new=1000; cursor_to=2025-10-15T08:01:22+00:00 raw_oldest=2024-05-15T20:09:00+00:00 oldest_new=2024-05-15T20:09:00+00:00
[AAPL][Batch 4] raw=1000, new=1000; cursor_to=2024-05-15T20:08:00+00:00 raw_oldest=2020-01-31T23:07:04+00:00 oldest_new=2020-01-31T23:07:04+00:00
[AAPL][Batch 5] feed=0; diag=No articles found. Please adjust the time range or refer to the API documentation https://www.alphavantage.co/documentation#newsapi and try again. raw_keys=['Information']
[AAPL] Saved 4000 rows → news_data/AAPL_20100101_20251209.xlsx


KeyboardInterrupt: 

In [ ]:
#Getting price data(minute level)
import os
import io
import time
import requests
import pandas as pd
from datetime import datetime, timezone

# ========= CONFIG =========
TICKERS = [
    "UNH","JNJ",
    "V","XOM","JPM","PG","MA","HD","CVX","LLY","ABBV","MRK",
    "PEP","KO","AVGO","COST","PFE","BAC","TMO","ADBE","DIS","CSCO",
    "WMT","NFLX","ABT","ACN","INTC","DHR","MCD","NKE","TXN","NEE",
    "VZ","LIN","PM","ORCL","UPS","BMY","HON","RTX","AMGN","IBM"
]
API_KEY = "MXZ437NRYQRY9FDY"  # <- replace
INTERVAL = "1min"                   # 1min / 5min / 15min / 30min / 60min
ADJUSTED = "true"                   # "true" (split/dividend adjusted) or "false"
EXTENDED_HOURS = "false"             # include pre/post market
OUTPUT_DIR = "price_data"           # each ticker -> one Excel
START_AT_STR = "2022-03-01 00:00:00Z"  # inclusive UTC start datetime
MIN_INTERVAL_S = 0.75               # ~75 rpm safety for Premium
MAX_RETRIES = 5                     # per-call retry attempts

# If your environment already defines TICKERS, we use it; otherwise a small fallback
try:
    TICKERS  # noqa
except NameError:
    TICKERS = ["AAPL", "MSFT", "NVDA", "GOOG", "AMZN"]

os.makedirs(OUTPUT_DIR, exist_ok=True)

SESSION = requests.Session()
BASE_URL = "https://www.alphavantage.co/query"

def parse_utc(s: str) -> datetime:
    """Parse a UTC datetime string; default to UTC if tz-naive."""
    fmts = ["%Y-%m-%d %H:%M:%SZ", "%Y-%m-%d %H:%M:%S", "%Y-%m-%d"]
    for f in fmts:
        try:
            dt = datetime.strptime(s.strip(), f)
            if dt.tzinfo is None:
                dt = dt.replace(tzinfo=timezone.utc)
            return dt.astimezone(timezone.utc)
        except Exception:
            pass
    raise ValueError(f"Cannot parse START_AT_STR: {s}")

def month_floor(dt: datetime) -> datetime:
    """Return the first moment of the month for dt (UTC)."""
    return dt.replace(day=1, hour=0, minute=0, second=0, microsecond=0)

def iter_months_backward(start_month_utc: datetime, end_month_utc: datetime):
    """
    Yield 'YYYY-MM' month strings from end_month down to start_month (inclusive).
    Example: ... 2025-11, 2025-10, ..., 2022-03
    """
    y, m = end_month_utc.year, end_month_utc.month
    ys, ms = start_month_utc.year, start_month_utc.month
    while (y, m) >= (ys, ms):
        yield f"{y:04d}-{m:02d}"
        if m == 1:
            y, m = y - 1, 12
        else:
            m -= 1

def normalize_intraday_columns(df: pd.DataFrame, symbol: str, month_yyyy_mm: str) -> pd.DataFrame:
    """
    Normalize CSV headers to a consistent schema: time, open, high, low, close, volume.
    Alpha Vantage sometimes uses 'timestamp' (or 'datetime'/'date') instead of 'time'.
    """
    if df.empty:
        return df

    # Lowercase and strip spaces
    df.columns = [c.strip().lower() for c in df.columns]

    # Map possible time column variations -> 'time'
    time_aliases = ["time", "timestamp", "datetime", "date"]
    time_col = next((c for c in time_aliases if c in df.columns), None)
    if time_col and time_col != "time":
        df = df.rename(columns={time_col: "time"})

    # Some sanity checks
    expected_core = {"time", "open", "high", "low", "close", "volume"}
    missing = [c for c in expected_core if c not in df.columns]
    if missing:
        # Print diagnostic and return empty to avoid KeyErrors downstream
        print(f"[{symbol}][{month_yyyy_mm}] Unexpected columns {list(df.columns)}; missing {missing}. Skipping this month.")
        return pd.DataFrame(columns=["time","open","high","low","close","volume"])

    # Keep only required columns & ensure dtypes
    df = df[["time","open","high","low","close","volume"]].copy()

    # Parse time column; Alpha Vantage intraday times are exchange-local (naive).
    df["time"] = pd.to_datetime(df["time"], errors="coerce")
    df = df.dropna(subset=["time"])

    # Coerce numeric columns
    for col in ["open","high","low","close","volume"]:
        df[col] = pd.to_numeric(df[col], errors="coerce")
    df = df.dropna(subset=["open","high","low","close","volume"])

    return df

def get_intraday_month_csv(symbol: str, month_yyyy_mm: str) -> pd.DataFrame:
    """
    Fetch one month of intraday data via TIME_SERIES_INTRADAY with 'month' and 'datatype=csv'.
    Returns a normalized DataFrame with columns: time, open, high, low, close, volume.
    """
    params = {
        "function": "TIME_SERIES_INTRADAY",
        "symbol": symbol,
        "interval": INTERVAL,
        "month": month_yyyy_mm,     # e.g., "2022-03"
        "outputsize": "full",       # full month when 'month' is specified
        "adjusted": ADJUSTED,
        "extended_hours": EXTENDED_HOURS,
        "datatype": "csv",          # CSV for speed and stability
        "apikey": API_KEY,
    }

    tries = 0
    while True:
        tries += 1
        resp = SESSION.get(BASE_URL, params=params, timeout=30)
        resp.raise_for_status()
        text = resp.text.strip()

        # If server returns JSON (rate-limit or info), it will start with '{'
        if text.startswith("{"):
            diag = resp.json()
            msg = diag.get("Note") or diag.get("Information") or diag.get("Error Message") or str(diag)
            if tries <= MAX_RETRIES and any(
                k in msg.lower() for k in ["frequency", "limit", "per minute", "per day", "call"]
            ):
                wait_s = min(4.0 * tries, 12.0)
                print(f"[{symbol}][{month_yyyy_mm}] Note: {msg} -> retry in {wait_s:.1f}s ({tries}/{MAX_RETRIES})")
                time.sleep(wait_s)
                continue
            print(f"[{symbol}][{month_yyyy_mm}] Error JSON: {msg}")
            return pd.DataFrame(columns=["time","open","high","low","close","volume"])

        # CSV path
        df_raw = pd.read_csv(io.StringIO(text))
        df_norm = normalize_intraday_columns(df_raw, symbol, month_yyyy_mm)
        return df_norm

def main():
    start_at = parse_utc(START_AT_STR)
    now_utc = datetime.now(timezone.utc)

    start_month = month_floor(start_at)
    end_month = month_floor(now_utc)

    print(f"Window: {start_at.isoformat()} -> {now_utc.isoformat()} (UTC)")
    print(f"Tickers: {len(TICKERS)} | Interval: {INTERVAL} | Adjusted: {ADJUSTED}")

    for sym in TICKERS:
        try:
            frames = []
            total_rows = 0

            # Walk from the most recent month backwards to the start month
            for mm in iter_months_backward(start_month, end_month):
                df = get_intraday_month_csv(sym, mm)
                time.sleep(MIN_INTERVAL_S)  # pacing
                if df.empty:
                    continue

                # Filter by the exact datetime window [start_at, now_utc]
                # df["time"] is naive local; compare with naive versions of our bounds
                df = df[(df["time"] >= start_at.replace(tzinfo=None)) & (df["time"] <= now_utc.replace(tzinfo=None))]

                frames.append(df)
                total_rows += len(df)
                print(f"[{sym}][{mm}] rows={len(df)} (acc={total_rows})")

            if not frames:
                print(f"[{sym}] No data in the requested window.")
                continue

            full = pd.concat(frames, axis=0, ignore_index=True)

            # Drop potential duplicates on timestamp
            full = full.drop_duplicates(subset=["time"], keep="first")

            # Sort with the most recent first
            full = full.sort_values("time", ascending=False).reset_index(drop=True)

            # Save one Excel per ticker
            fname = f"{sym}_{INTERVAL}_{start_at.strftime('%Y%m%d')}_{now_utc.strftime('%Y%m%d')}.xlsx"
            fpath = os.path.join(OUTPUT_DIR, fname)
            full.to_excel(fpath, index=False)
            print(f"[{sym}] Saved {len(full):,} rows -> {fpath}")

        except Exception as e:
            print(f"[{sym}] Failed: {e}")

if __name__ == "__main__":
    main()


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import numpy as np
news_df = pd.read_parquet("/content/drive/MyDrive/COMS Shared Data/news_df.parquet")
price_df = pd.read_parquet("/content/drive/MyDrive/COMS Shared Data/price_df.parquet")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel
import numpy as np
import pandas as pd
import re

device = "cuda" if torch.cuda.is_available() else "cpu"

df = news_df.copy()
df = df.dropna(subset=["summary", "authority_level"])

def clean_summary(s):
    s = str(s)
    s = re.sub(r"<[^>]+>", " ", s)
    s = re.sub(r"\s+", " ", s)
    if len(s) > 2000:
        s = s[:2000]
    return s.strip()

df["summary_clean"] = df["summary"].apply(clean_summary)
df["label"] = (df["authority_level"] == 3).astype(float)

np.random.seed(42)
indices = np.arange(len(df))
np.random.shuffle(indices)
split = int(0.9 * len(indices))
train_idx, val_idx = indices[:split], indices[split:]

train_df = df.iloc[train_idx].reset_index(drop=True)
val_df   = df.iloc[val_idx].reset_index(drop=True)

model_name = "ProsusAI/finbert"
tokenizer = AutoTokenizer.from_pretrained(model_name)

class NewsDataset(Dataset):
    def __init__(self, dataframe):
        self.texts = list(dataframe["summary_clean"])
        self.labels = dataframe["label"].astype(float).values

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            "text": self.texts[idx],
            "label": float(self.labels[idx]),
        }

def collate_fn(batch):
    texts  = [b["text"] for b in batch]
    labels = torch.tensor([b["label"] for b in batch], dtype=torch.float)
    enc = tokenizer(texts, padding=True, truncation=True, max_length=256, return_tensors="pt")
    enc["labels"] = labels
    return enc

train_ds = NewsDataset(train_df)
val_ds   = NewsDataset(val_df)

train_loader = DataLoader(train_ds, batch_size=16, shuffle=True, collate_fn=collate_fn)
val_loader   = DataLoader(val_ds, batch_size=32, shuffle=False, collate_fn=collate_fn)

encoder = AutoModel.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [ ]:
from tqdm import tqdm

class FilteringModel(nn.Module):
    def __init__(self, encoder):
        super().__init__()
        self.encoder = encoder
        hidden_size = encoder.config.hidden_size
        self.head = nn.Linear(hidden_size, 1)

    def forward(self, input_ids, attention_mask):
        out = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
        cls_hidden = out.last_hidden_state[:, 0, :]
        logits = self.head(cls_hidden).squeeze(-1)
        return logits


In [ ]:
#Train model to filter good quality news
model = FilteringModel(encoder).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

# ------------------------
# Train + Validation
# ------------------------
EPOCHS = 10

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0.0
    n_batches = 0

    print(f"\n🔵 Epoch {epoch+1}/{EPOCHS} — Training")
    for batch in tqdm(train_loader, desc="Training", leave=False):
        ids  = batch["input_ids"].to(device)
        mask = batch["attention_mask"].to(device)
        lbl  = batch["labels"].to(device)

        logits = model(ids, mask)
        loss = criterion(logits, lbl)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        n_batches += 1

    avg_train_loss = total_loss / max(1, n_batches)

    # -------- Validation --------
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    print(f"🟣 Epoch {epoch+1}/{EPOCHS} — Validation")
    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Validating", leave=False):
            ids  = batch["input_ids"].to(device)
            mask = batch["attention_mask"].to(device)
            lbl  = batch["labels"].to(device)

            logits = model(ids, mask)
            loss = criterion(logits, lbl)
            val_loss += loss.item()

            probs = torch.sigmoid(logits)
            preds = (probs >= 0.5).float()
            val_correct += (preds == lbl).sum().item()
            val_total   += lbl.numel()

    avg_val_loss = val_loss / max(1, len(val_loader))
    val_acc = val_correct / max(1, val_total)

    print(
        f"Epoch {epoch+1}/{EPOCHS} | "
        f"train_loss = {avg_train_loss:.4f} | "
        f"val_loss = {avg_val_loss:.4f} | "
        f"val_acc = {val_acc:.4f}"
    )



🔵 Epoch 1/10 — Training


KeyboardInterrupt: 

In [ ]:
encoder = AutoModel.from_pretrained("bert-base-uncased")
model = FilteringModel(encoder).to(device)
model_path = "/content/drive/MyDrive/COMS Shared Data/news_filter_model.pt"
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()
print("Model loaded!")


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Model loaded!


In [ ]:
full_ds = NewsDataset(df)
full_loader = DataLoader(full_ds, batch_size=32, shuffle=False, collate_fn=collate_fn)

model.eval()
all_scores = []

print("\n🟢 Scoring all news rows…")
with torch.no_grad():
    for batch in tqdm(full_loader, desc="Scoring", leave=True):
        ids  = batch["input_ids"].to(device)
        mask = batch["attention_mask"].to(device)
        logits = model(ids, mask)
        probs = torch.sigmoid(logits)
        all_scores.append(probs.cpu())

all_scores = torch.cat(all_scores).numpy()
df["quality_score_model"] = all_scores

news_df.loc[df.index, "quality_score_model"] = df["quality_score_model"]

news_df_sorted = news_df.sort_values("quality_score_model", ascending=False)



🟢 Scoring all news rows…


Scoring:   2%|▏         | 101/5704 [00:06<06:07, 15.26it/s]


KeyboardInterrupt: 

In [ ]:
news_sorted = pd.read_parquet("/content/drive/MyDrive/COMS Shared Data/news_sorted.parquet")
news_filtered = news_sorted[news_sorted["quality_score_model"] > 0.05]

In [ ]:
import pandas as pd
import numpy as np

print("Step 1: copy and basic type conversion")
df = price_df.copy()
df["time"] = pd.to_datetime(df["time"])

for col in ["open", "high", "low", "close"]:
    df[col] = pd.to_numeric(df[col], downcast="float")
df["volume"] = pd.to_numeric(df["volume"], downcast="integer")

print("Step 2: extract ticker from ticker_file")
df["ticker"] = df["ticker_file"].str.extract(r"^([^_]+)_", expand=False).astype("category")

print("Step 3: sort by ticker and time")
df = df.sort_values(["ticker", "time"])
df["date"] = df["time"].dt.date

print("Step 4: compute forward 5-minute close inside each ticker-day")
df["close_fwd5"] = df.groupby(["ticker", "date"], observed=True)["close"].shift(-4)

print("Step 5: compute next-day open for each ticker-date")
daily_open = df.groupby(["ticker", "date"], observed=True)["open"].first()
next_day_open = daily_open.groupby(level=0).shift(-1).rename("next_day_open")
df = df.join(next_day_open, on=["ticker", "date"])

print("Step 6: identify last 5 minutes of each day")
rank_desc = df.groupby(["ticker", "date"])["time"].rank(method="first", ascending=False)
last5_mask = rank_desc <= 5

print("Step 7: build forward price using 5min close or next-day open")
df["fwd_price"] = df["close_fwd5"]
mask = last5_mask & df["close_fwd5"].isna() & df["next_day_open"].notna()
df.loc[mask, "fwd_price"] = df.loc[mask, "next_day_open"]

print("Step 8: compute return and build ret_df")
df["ret_5m"] = df["fwd_price"] / df["open"] - 1
ret_df = df[df["fwd_price"].notna()][["ticker", "time", "ret_5m"]]

print("Done. Shape of ret_df:", ret_df.shape)
print(ret_df.head())

ret_df.to_parquet("/content/drive/MyDrive/COMS Shared Data/ret_df.parquet")


Step 1: copy and basic type conversion


NameError: name 'price_df' is not defined

In [ ]:
ret_df = pd.read_parquet("/content/drive/MyDrive/COMS Shared Data/ret_df.parquet")

In [ ]:
#combine news with log return
import pandas as pd
import numpy as np
from datetime import time

px = price_df[["time", "open", "close", "ticker_file"]].copy()
px["time"] = pd.to_datetime(px["time"])
px["ticker"] = px["ticker_file"].str.extract(r"^([^_]+)_", expand=False).astype(str)

px = px.sort_values(["ticker", "time"])
px["date"] = px["time"].dt.normalize()

daily = px.groupby(["ticker", "date"], observed=True).agg(
    first_open=("open", "first"),
    last_close=("close", "last")
).reset_index()

daily["prev_last_close"] = daily.groupby("ticker")["last_close"].shift(1)
daily["overnight_ret"] = daily["first_open"] / daily["prev_last_close"] - 1

overnight_df = daily.dropna(subset=["overnight_ret"])[["ticker", "date", "overnight_ret"]]
overnight_df = overnight_df.rename(columns={"date": "session_date"})

trading_days = np.sort(overnight_df["session_date"].unique())

news = news_filtered.copy()
news["time"] = pd.to_datetime(news["time"])
if getattr(news["time"].dtype, "tz", None) is not None:
    news["time"] = news["time"].dt.tz_convert(None)

news["ticker"] = news["ticker"].astype(str)
ret_df["ticker"] = ret_df["ticker"].astype(str)

market_open = time(9, 30)
market_close = time(15, 59)
tod = news["time"].dt.time
news["date"] = news["time"].dt.normalize()
is_trading_day = news["date"].isin(trading_days)

news["in_session"] = (
    (tod >= market_open) & (tod <= market_close) & is_trading_day
)

news["trade_time"] = news["time"].dt.floor("min") + pd.Timedelta(minutes=1)

ret_tmp = (
    ret_df[["ticker", "time", "ret_5m"]]
    .rename(columns={"time": "trade_time", "ret_5m": "ret_5m_after_news"})
)

news = news.merge(ret_tmp, on=["ticker", "trade_time"], how="left")

news["session_date_candidate"] = pd.NaT

mask_pre  = (~news["in_session"]) & (tod < market_open)
mask_post = (~news["in_session"]) & (tod > market_close)
mask_other = (~news["in_session"]) & ~(mask_pre | mask_post)

news.loc[mask_pre,  "session_date_candidate"] = news.loc[mask_pre,  "date"]
news.loc[mask_post, "session_date_candidate"] = news.loc[mask_post, "date"] + pd.Timedelta(days=1)
news.loc[mask_other, "session_date_candidate"] = news.loc[mask_other, "date"]

news["session_date"] = pd.NaT
mask_cand = news["session_date_candidate"].notna()
cand = news.loc[mask_cand, "session_date_candidate"].values.astype("datetime64[ns]")

idx = trading_days.searchsorted(cand, side="left")
valid = idx < len(trading_days)

mapped = np.full(cand.shape, np.datetime64("NaT"), dtype="datetime64[ns]")
mapped[valid] = trading_days[idx[valid]]

news.loc[mask_cand, "session_date"] = mapped

news = news.merge(overnight_df, on=["ticker", "session_date"], how="left")

news["ret_after_news"] = np.where(
    news["in_session"],
    news["ret_5m_after_news"],
    news["overnight_ret"]
)

news_with_ret = news
print("NA count:", news_with_ret["ret_after_news"].isna().sum())
news_with_ret.to_parquet("/content/drive/MyDrive/COMS Shared Data/news_with_ret.parquet")


KeyboardInterrupt: 

In [ ]:
# ==========================
# Cell 1: common config
# ==========================
import os
import re
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from tqdm.auto import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"
BASE_DIR = "/content/drive/MyDrive/COMS Shared Data"

FINBERT_NAME = "ProsusAI/finbert"
cls_tokenizer = AutoTokenizer.from_pretrained(FINBERT_NAME)


In [ ]:
# ======================================================
# Cell 2: build 8-class dataset from news_with_ret_5min
# ======================================================

def clean_summary_for_cls(s: str) -> str:
    """Simple text cleaning for classification."""
    s = str(s)
    s = re.sub(r"<[^>]+>", " ", s)
    s = re.sub(r"\s+", " ", s)
    if len(s) > 2000:
        s = s[:2000]
    return s.strip()

news_5m_path = os.path.join(BASE_DIR, "news_with_ret_5min.parquet")
news_5m = pd.read_parquet(news_5m_path)

# We only keep rows with valid 5-minute return after news
news_5m = news_5m.dropna(subset=["ret_after_news_5m"])
news_5m = news_5m[np.isfinite(news_5m["ret_after_news_5m"])].copy()

# Log return of (1 + ret_after_news_5m)
news_5m["r_5m"] = 1.0 + news_5m["ret_after_news_5m"]
news_5m = news_5m[news_5m["r_5m"] > 1e-8].copy()
news_5m["log_ret_5m"] = np.log(news_5m["r_5m"])

# Remove extreme tails for stability
q_low, q_high = news_5m["log_ret_5m"].quantile([0.01, 0.99])
news_5m = news_5m[news_5m["log_ret_5m"].between(q_low, q_high)].copy()

# 8-class label by quantiles
num_bins_7 = 7
news_5m["label_7cls"] = pd.qcut(
    news_5m["log_ret_5m"],
    q=num_bins_7,
    labels=False,
    duplicates="drop",
).astype(int)

label_names_7 = [
    "very_strong_down",
    "strong_down",
    "moderate_down",
    "flat",
    "moderate_up",
    "strong_up",
    "very_strong_up",
]

# Text feature: ticker + cleaned summary
news_5m["summary_clean_cls"] = news_5m["summary"].apply(clean_summary_for_cls)
news_5m["text"] = news_5m["ticker"].astype(str) + " [SEP] " + news_5m["summary_clean_cls"]

# Train/val/test split (stratified by 8-class label)
train_df_7, temp_df_7 = train_test_split(
    news_5m,
    test_size=0.2,
    random_state=42,
    stratify=news_5m["label_7cls"],
)
val_df_7, test_df_7 = train_test_split(
    temp_df_7,
    test_size=0.5,
    random_state=43,
    stratify=temp_df_7["label_7cls"],
)

print("7-class splits | train / val / test:", len(train_df_7), len(val_df_7), len(test_df_7))


7-class splits | train / val / test: 17473 2184 2185


In [ ]:
# ======================================================
# Cell 3: generic FinBERT classifier utilities
# ======================================================

class ReturnDataset(Dataset):
    """Generic dataset for classification on news text."""

    def __init__(self, df: pd.DataFrame, text_col: str, label_col: str):
        self.texts = df[text_col].tolist()
        self.labels = df[label_col].astype(int).tolist()

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            "text": self.texts[idx],
            "label": self.labels[idx],
        }

def cls_collate_fn(batch):
    """Tokenize a batch of texts and pack labels."""
    texts = [b["text"] for b in batch]
    labels = torch.tensor([b["label"] for b in batch], dtype=torch.long)
    enc = cls_tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=256,
        return_tensors="pt",
    )
    enc["labels"] = labels
    return enc

class ReturnClassifier(nn.Module):
    """
    FinBERT sequence classification model for multi-class tasks.
    """

    def __init__(self, num_labels: int):
        super().__init__()
        self.model = AutoModelForSequenceClassification.from_pretrained(
            FINBERT_NAME,
            num_labels=num_labels,
            ignore_mismatched_sizes=True
        )

    def forward(self, input_ids=None, attention_mask=None, labels=None):
        return self.model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels,
        )

def evaluate_classifier(model, data_loader, label_names=None):
    """Evaluate model on a dataloader and print metrics."""
    model.eval()
    total_loss, total_correct, total_examples = 0.0, 0, 0
    all_labels, all_preds = [], []

    with torch.no_grad():
        for batch in tqdm(data_loader, desc="Eval", leave=False):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels,
            )
            loss = outputs.loss
            logits = outputs.logits

            total_loss += loss.item() * labels.size(0)
            preds = logits.argmax(dim=-1)
            total_correct += (preds == labels).sum().item()
            total_examples += labels.size(0)

            all_labels.extend(labels.cpu().numpy().tolist())
            all_preds.extend(preds.cpu().numpy().tolist())

    avg_loss = total_loss / total_examples if total_examples > 0 else 0.0
    acc = total_correct / total_examples if total_examples > 0 else 0.0

    print(f"Eval loss={avg_loss:.4f}, acc={acc:.4f}")
    if label_names is not None:
        print(classification_report(all_labels, all_preds, target_names=label_names, digits=4))

    return avg_loss, acc

def train_classifier(
    model,
    train_loader,
    val_loader,
    num_epochs: int = 5,
    lr: float = 2e-5,
    label_names=None,
):
    """Standard training loop for classification models."""
    model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

    best_val_acc = 0.0
    best_state = None

    for epoch in range(num_epochs):
        # ----- Train -----
        model.train()
        total_loss, total_correct, total_examples = 0.0, 0, 0

        for batch in tqdm(train_loader, desc=f"Train epoch {epoch+1}/{num_epochs}", leave=False):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            optimizer.zero_grad()
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels,
            )
            loss = outputs.loss
            logits = outputs.logits

            loss.backward()
            optimizer.step()

            total_loss += loss.item() * labels.size(0)
            preds = logits.argmax(dim=-1)
            total_correct += (preds == labels).sum().item()
            total_examples += labels.size(0)

        train_loss = total_loss / total_examples
        train_acc = total_correct / total_examples
        print(f"[Epoch {epoch+1}] train_loss={train_loss:.4f}, train_acc={train_acc:.4f}")

        # ----- Validation -----
        val_loss, val_acc = evaluate_classifier(model, val_loader, label_names=label_names)

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_state = model.state_dict()

    if best_state is not None:
        model.load_state_dict(best_state)
    print(f"Best val acc = {best_val_acc:.4f}")
    return model


In [ ]:
# ======================================================
# Cell 4: train 8-class 5-minute FinBERT baseline
# ======================================================

train_ds_7 = ReturnDataset(train_df_7, text_col="text", label_col="label_7cls")
val_ds_7   = ReturnDataset(val_df_7,   text_col="text", label_col="label_7cls")
test_ds_7  = ReturnDataset(test_df_7,  text_col="text", label_col="label_7cls")

train_loader_7 = DataLoader(train_ds_7, batch_size=16, shuffle=True,  collate_fn=cls_collate_fn)
val_loader_7   = DataLoader(val_ds_7,   batch_size=32, shuffle=False, collate_fn=cls_collate_fn)
test_loader_7  = DataLoader(test_ds_7,  batch_size=32, shuffle=False, collate_fn=cls_collate_fn)

model_7cls = ReturnClassifier(num_labels=len(label_names_7))
model_7cls = train_classifier(
    model_7cls,
    train_loader_7,
    val_loader_7,
    num_epochs=5,
    lr=2e-5,
    label_names=label_names_7,
)

print("=== 7-class 5-minute test performance ===")
evaluate_classifier(model_7cls, test_loader_7, label_names=label_names_7)

model_7_path = os.path.join(BASE_DIR, "finbert_7cls_5min.pt")
torch.save(model_7cls.state_dict(), model_7_path)
print("Saved 7-class 5-minute model to", model_7_path)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ProsusAI/finbert and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([7, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([7]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Train epoch 1/5:   0%|          | 0/1093 [00:00<?, ?it/s]

[Epoch 1] train_loss=1.9385, train_acc=0.1695


Eval:   0%|          | 0/69 [00:00<?, ?it/s]

Eval loss=1.9051, acc=0.1983
                  precision    recall  f1-score   support

very_strong_down     0.2321    0.0833    0.1226       312
     strong_down     0.1167    0.0224    0.0376       312
   moderate_down     0.3043    0.0224    0.0418       312
            flat     0.1798    0.5705    0.2734       312
     moderate_up     0.3333    0.0032    0.0063       312
       strong_up     0.1663    0.2244    0.1910       312
  very_strong_up     0.2504    0.4615    0.3247       312

        accuracy                         0.1983      2184
       macro avg     0.2261    0.1983    0.1425      2184
    weighted avg     0.2261    0.1983    0.1425      2184



...

[Epoch 5] train_loss=1.2042, train_acc=0.5549


Eval:   0%|          | 0/69 [00:00<?, ?it/s]

Eval loss=2.4077, acc=0.2015
                  precision    recall  f1-score   support

very_strong_down     0.2591    0.2724    0.2656       312
     strong_down     0.1860    0.1442    0.1625       312
   moderate_down     0.1419    0.1378    0.1398       312
            flat     0.1662    0.1859    0.1755       312
     moderate_up     0.1862    0.2853    0.2253       312
       strong_up     0.1607    0.0577    0.0849       312
  very_strong_up     0.2742    0.3269    0.2982       312

        accuracy                         0.2015      2184
       macro avg     0.1963    0.2015    0.1931      2184
    weighted avg     0.1963    0.2015    0.1931      2184

Best val acc = 0.2189
=== 7-class 5-minute test performance ===


Eval:   0%|          | 0/69 [00:00<?, ?it/s]

Eval loss=2.4089, acc=0.2114
                  precision    recall  f1-score   support

very_strong_down     0.2471    0.2748    0.2602       313
     strong_down     0.1818    0.1278    0.1501       313
   moderate_down     0.2107    0.1897    0.1997       311
            flat     0.2262    0.2436    0.2346       312
     moderate_up     0.1824    0.2724    0.2185       312
       strong_up     0.2158    0.0962    0.1330       312
  very_strong_up     0.2172    0.2756    0.2429       312

        accuracy                         0.2114      2185
       macro avg     0.2116    0.2114    0.2056      2185
    weighted avg     0.2116    0.2114    0.2056      2185

Saved 7-class 5-minute model to /content/drive/MyDrive/COMS Shared Data/finbert_7cls_5min.pt


In [ ]:
# ======================================================
# Cell M1 (optional): build intraday returns for {1,5,30,60} minutes
# ======================================================
import numpy as np
import pandas as pd
import os

HORIZONS_ALL = [1, 5, 30, 60]

def build_intraday_ret(price_df: pd.DataFrame, horizon_minutes: int, save_path: str | None = None) -> str:
    """
    Build intraday forward returns for a given horizon (in minutes) and save to parquet.

    Output columns: ticker, time, ret_{horizon}m
    File name: ret_df_{horizon}min.parquet (compatible with your original code).
    """
    df = price_df.copy()
    df["time"] = pd.to_datetime(df["time"])

    # Basic type conversion
    for col in ["open", "high", "low", "close"]:
        df[col] = pd.to_numeric(df[col], downcast="float")
    df["volume"] = pd.to_numeric(df["volume"], downcast="integer")

    # Extract ticker from ticker_file
    df["ticker"] = df["ticker_file"].str.extract(r"^([^_]+)_", expand=False).astype("category")
    df = df.sort_values(["ticker", "time"])
    df["date"] = df["time"].dt.date

    steps = int(horizon_minutes)

    # Forward close inside each ticker-day
    df[f"close_fwd{horizon_minutes}"] = (
        df.groupby(["ticker", "date"], observed=True)["close"].shift(-steps)
    )

    # Next-day open for last minutes of the day
    daily_open = df.groupby(["ticker", "date"], observed=True)["open"].first()
    next_day_open = daily_open.groupby(level=0).shift(-1).rename("next_day_open")
    df = df.join(next_day_open, on=["ticker", "date"])

    # Identify last {steps} minutes of each day
    rank_desc = df.groupby(["ticker", "date"])["time"].rank(method="first", ascending=False)
    last_mask = rank_desc <= steps

    # Build forward price using horizon close or next-day open
    df["fwd_price"] = df[f"close_fwd{horizon_minutes}"]
    mask = last_mask & df[f"close_fwd{horizon_minutes}"].isna() & df["next_day_open"].notna()
    df.loc[mask, "fwd_price"] = df.loc[mask, "next_day_open"]

    # Forward return
    df[f"ret_{horizon_minutes}m"] = df["fwd_price"] / df["open"] - 1.0
    ret_df = df[df["fwd_price"].notna()][["ticker", "time", f"ret_{horizon_minutes}m"]].copy()

    if save_path is None:
        save_path = os.path.join(BASE_DIR, f"ret_df_{horizon_minutes}min.parquet")

    ret_df.to_parquet(save_path)
    print("saved intraday ret:", save_path, "shape:", ret_df.shape)
    return save_path

ret_paths = {h: build_intraday_ret(price_df, h) for h in HORIZONS_ALL}
print("all intraday ret paths:", ret_paths)


In [ ]:
# ======================================================
# Cell M2 (optional): build overnight returns and news_with_ret_{h}min
# ======================================================
from datetime import time, timedelta
import numpy as np

def compute_overnight_returns(price_df: pd.DataFrame, save_path: str | None = None) -> pd.DataFrame:
    """
    Compute overnight return:
        overnight_ret = next day's open / previous day's last close - 1
    Output columns: ticker, session_date, overnight_ret
    """
    px = price_df[["time", "open", "close", "ticker_file"]].copy()
    px["time"] = pd.to_datetime(px["time"])
    px["ticker"] = px["ticker_file"].str.extract(r"^([^_]+)_", expand=False).astype(str)

    px = px.sort_values(["ticker", "time"])
    px["date"] = px["time"].dt.normalize()

    daily = px.groupby(["ticker", "date"], observed=True).agg(
        first_open=("open", "first"),
        last_close=("close", "last"),
    ).reset_index()

    daily["prev_last_close"] = daily.groupby("ticker")["last_close"].shift(1)
    daily["overnight_ret"] = daily["first_open"] / daily["prev_last_close"] - 1.0

    overnight_df = daily.dropna(subset=["overnight_ret"])[["ticker", "date", "overnight_ret"]].copy()
    overnight_df = overnight_df.rename(columns={"date": "session_date"})

    if save_path is None:
        save_path = os.path.join(BASE_DIR, "overnight_ret.parquet")
    overnight_df.to_parquet(save_path)
    print("saved overnight:", save_path, "shape:", overnight_df.shape)
    return overnight_df

overnight_path = os.path.join(BASE_DIR, "overnight_ret.parquet")
overnight_df = compute_overnight_returns(price_df, overnight_path)
trading_days = np.sort(overnight_df["session_date"].unique())

def build_news_with_ret_for_horizon(
    news_base: pd.DataFrame,
    horizon_minutes: int,
    trading_days: np.ndarray,
    overnight_df: pd.DataFrame,
    base_dir: str,
    save_path: str | None = None,
) -> str:
    """
    For a given horizon, combine intraday return and overnight return to get:
        ret_after_news_{horizon}m

    If news is in session: use intraday return after news.
    Otherwise: use overnight return to the next trading day.
    """
    news = news_base.copy()
    news["time"] = pd.to_datetime(news["time"])
    if getattr(news["time"].dtype, "tz", None) is not None:
        news["time"] = news["time"].dt.tz_convert(None)
    news["ticker"] = news["ticker"].astype(str)

    # Load intraday returns
    ret_df = pd.read_parquet(os.path.join(base_dir, f"ret_df_{horizon_minutes}min.parquet"))
    ret_df["time"] = pd.to_datetime(ret_df["time"])
    ret_df["ticker"] = ret_df["ticker"].astype(str)

    # Session / non-session logic
    market_open = time(9, 30)
    market_close = time(15, 59)
    tod = news["time"].dt.time
    news["date"] = news["time"].dt.normalize()

    is_trading_day = news["date"].isin(trading_days)

    news["in_session"] = (
        (tod >= market_open) &
        (tod <= market_close) &
        is_trading_day
    )

    # Intraday return anchor: 1 minute after news
    news["trade_time"] = news["time"].dt.floor("min") + pd.Timedelta(minutes=1)

    col_intraday = f"ret_{horizon_minutes}m"
    col_after    = f"ret_after_news_{horizon_minutes}m"

    ret_tmp = (
        ret_df[["ticker", "time", col_intraday]]
        .rename(columns={"time": "trade_time", col_intraday: f"{col_after}_intraday"})
    )

    news = news.merge(ret_tmp, on=["ticker", "trade_time"], how="left")

    # Map news timestamp to the corresponding trading session
    news["session_date_candidate"] = pd.NaT

    mask_pre  = (~news["in_session"]) & (tod < market_open)
    mask_post = (~news["in_session"]) & (tod > market_close)
    mask_other = (~news["in_session"]) & ~(mask_pre | mask_post)

    news.loc[mask_pre,  "session_date_candidate"] = news.loc[mask_pre,  "date"]
    news.loc[mask_post, "session_date_candidate"] = news.loc[mask_post, "date"] + pd.Timedelta(days=1)
    news.loc[mask_other, "session_date_candidate"] = news.loc[mask_other, "date"]

    news["session_date"] = pd.NaT
    mask_cand = news["session_date_candidate"].notna()
    cand = news.loc[mask_cand, "session_date_candidate"].values.astype("datetime64[ns]")

    idx = trading_days.searchsorted(cand, side="left")
    valid = idx < len(trading_days)

    mapped = np.full(cand.shape, np.datetime64("NaT"), dtype="datetime64[ns]")
    mapped[valid] = trading_days[idx[valid]]

    news.loc[mask_cand, "session_date"] = mapped

    # Merge overnight returns
    news = news.merge(overnight_df, on=["ticker", "session_date"], how="left")

    # Combine intraday and overnight
    news[col_after] = np.where(
        news["in_session"],
        news[f"{col_after}_intraday"],
        news["overnight_ret"],
    )

    news = news.drop(columns=[f"{col_after}_intraday"])

    if save_path is None:
        save_path = os.path.join(base_dir, f"news_with_ret_{horizon_minutes}min.parquet")
    news.to_parquet(save_path)
    print("saved", save_path, "shape:", news.shape, "NaN in", col_after, "=", news[col_after].isna().sum())
    return save_path

HORIZONS_ALL = [1, 5, 30, 60]

news_with_ret_paths = {}
for h in HORIZONS_ALL:
    # news_filtered should already exist in memory, or be loaded from news_filtered.parquet
    p = build_news_with_ret_for_horizon(news_filtered, h, trading_days, overnight_df, BASE_DIR)
    news_with_ret_paths[h] = p

print("all news_with_ret paths:", news_with_ret_paths)

# Just to check shapes of return files (optional)
ret_dfs = {h: pd.read_parquet(os.path.join(BASE_DIR, f"ret_df_{h}min.parquet")) for h in HORIZONS_ALL}
for h in HORIZONS_ALL:
    print(h, "min:", ret_dfs[h].shape)


In [ ]:
# ======================================================
# Cell M3 (updated): build labels for each horizon / class setting
# ======================================================
from sklearn.model_selection import train_test_split

def build_labels_for_horizon(df: pd.DataFrame, ret_col: str, n_classes: int):
    """
    Build classification labels from ret_after_news_{horizon}m.

    n_classes = 2:
        label_cls = 0 if ret <= 0, 1 if ret > 0
    n_classes = 3:
        label_cls in {0,1,2} by tertiles of log(1 + ret)
    n_classes = 7:
        label_cls in {0..6} by 1/7,2/7,...,6/7 quantiles of log(1 + ret)
    """
    df = df.dropna(subset=[ret_col]).copy()
    df[ret_col] = df[ret_col].astype("float64")

    # Avoid log(1 + r) issues
    df = df[df[ret_col] > -0.99].reset_index(drop=True)

    if n_classes == 2:
        df["label_cls"] = (df[ret_col] > 0).astype("int64")
        label_names = ["non_positive", "positive"]

    elif n_classes == 3:
        log_ret = np.log1p(df[ret_col])
        q1, q2 = log_ret.quantile([1.0 / 3.0, 2.0 / 3.0])
        bins = [-np.inf, q1, q2, np.inf]
        df["label_cls"] = np.digitize(log_ret, bins) - 1
        label_names = ["decrease", "flat", "increase"]

    elif n_classes == 7:
        # 7-class symmetric buckets based on quantiles of log(1 + ret)
        log_ret = np.log1p(df[ret_col])
        qs = [log_ret.quantile(k / 7.0) for k in range(1, 7)]  # 1/7,...,6/7
        bins = [-np.inf] + qs + [np.inf]
        df["label_cls"] = np.digitize(log_ret, bins) - 1
        label_names = [
            "strong_decrease",
            "moderate_decrease",
            "slight_decrease",
            "flat",
            "slight_increase",
            "moderate_increase",
            "strong_increase",
        ]
    else:
        raise ValueError("n_classes must be 2, 3, or 7")

    return df, label_names

def prepare_horizon_dataset(horizon_minutes: int, n_classes: int):
    """
    Load news_with_ret_{horizon}min.parquet, build labels and text column, split train/val/test.
    Reuse the same text style as 5min 8-class: "ticker [SEP] summary_clean".
    """
    news_path = os.path.join(BASE_DIR, f"news_with_ret_{horizon_minutes}min.parquet")
    df = pd.read_parquet(news_path)

    ret_col = f"ret_after_news_{horizon_minutes}m"
    df, label_names = build_labels_for_horizon(df, ret_col, n_classes)

    # Reuse cleaning function from 8-class section
    df["summary_clean_cls"] = df["summary"].apply(clean_summary_for_cls)
    df["text"] = df["ticker"].astype(str) + " [SEP] " + df["summary_clean_cls"]

    train_df, temp_df = train_test_split(
        df,
        test_size=0.2,
        random_state=42,
        stratify=df["label_cls"],
    )
    val_df, test_df = train_test_split(
        temp_df,
        test_size=0.5,
        random_state=43,
        stratify=temp_df["label_cls"],
    )

    print(
        f"H={horizon_minutes}min, C={n_classes}: "
        f"train/val/test = {len(train_df)}/{len(val_df)}/{len(test_df)}"
    )

    return train_df, val_df, test_df, label_names


In [ ]:
# ======================================================
# Cell M4 (updated): train FinBERT for multiple horizons and class counts
# ======================================================

def train_finbert_for_horizon(horizon_minutes: int, n_classes: int, epochs: int = 5):
    """
    Train a FinBERT classifier for a given horizon and number of classes.

    Saves:
        finbert_cls_{horizon}min_{n_classes}cls.pt
        finbert_cls_{horizon}min_{n_classes}cls_meta.parquet
    """
    assert n_classes in {2, 3, 7}, "n_classes must be 2, 3, or 7"

    train_df, val_df, test_df, label_names = prepare_horizon_dataset(horizon_minutes, n_classes)

    train_ds = ReturnDataset(train_df, text_col="text", label_col="label_cls")
    val_ds   = ReturnDataset(val_df,   text_col="text", label_col="label_cls")
    test_ds  = ReturnDataset(test_df,  text_col="text", label_col="label_cls")

    train_loader = DataLoader(train_ds, batch_size=16, shuffle=True,  collate_fn=cls_collate_fn)
    val_loader   = DataLoader(val_ds,   batch_size=32, shuffle=False, collate_fn=cls_collate_fn)
    test_loader  = DataLoader(test_ds,  batch_size=32, shuffle=False, collate_fn=cls_collate_fn)

    model = ReturnClassifier(num_labels=n_classes)
    model = train_classifier(
        model,
        train_loader,
        val_loader,
        num_epochs=epochs,
        lr=2e-5,
        label_names=label_names,
    )

    print(f"\n=== Test performance: H={horizon_minutes}min, C={n_classes} ===")
    evaluate_classifier(model, test_loader, label_names=label_names)

    # Save model and label metadata
    model_path = os.path.join(BASE_DIR, f"finbert_cls_{horizon_minutes}min_{n_classes}cls.pt")
    torch.save(model.state_dict(), model_path)

    meta = pd.DataFrame(
        {"label_id": list(range(len(label_names))), "label_name": label_names}
    )
    meta_path = os.path.join(BASE_DIR, f"finbert_cls_{horizon_minutes}min_{n_classes}cls_meta.parquet")
    meta.to_parquet(meta_path)

    print("saved model:", model_path)
    print("saved meta :", meta_path)

    return model_path, meta_path

# You already have a dedicated 5min 8-class model above.
# Here we experiment with other horizons {1,30,60} and class counts {2,3,7}.
HORIZONS_EXP = [1, 5, 30, 60]
CLASS_NUMS   = [2, 3, 7]

model_paths = {}
for h in HORIZONS_EXP:
    for c in CLASS_NUMS:
        mp, mp_meta = train_finbert_for_horizon(h, c, epochs=5)
        model_paths[(h, c)] = (mp, mp_meta)

print("all horizon/class models:", model_paths)


H=1min, C=2: train/val/test = 17825/2228/2229


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ProsusAI/finbert and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Train epoch 1/5:   0%|          | 0/1115 [00:00<?, ?it/s]

[Epoch 1] train_loss=0.6988, train_acc=0.5073


Eval:   0%|          | 0/70 [00:00<?, ?it/s]

...

[Epoch 5] train_loss=1.3239, train_acc=0.4895


Eval:   0%|          | 0/70 [00:00<?, ?it/s]

Eval loss=2.4347, acc=0.2033
                   precision    recall  f1-score   support

  strong_decrease     0.2536    0.2799    0.2661       318
moderate_decrease     0.1667    0.1792    0.1727       318
  slight_decrease     0.1891    0.2830    0.2267       318
             flat     0.1977    0.1101    0.1414       318
  slight_increase     0.1905    0.0502    0.0794       319
moderate_increase     0.1574    0.1693    0.1631       319
  strong_increase     0.2462    0.3522    0.2898       318

         accuracy                         0.2033      2228
        macro avg     0.2002    0.2034    0.1913      2228
     weighted avg     0.2001    0.2033    0.1913      2228

Best val acc = 0.2127

=== Test performance: H=60min, C=7 ===


Eval:   0%|          | 0/70 [00:00<?, ?it/s]

Eval loss=2.4368, acc=0.2135
                   precision    recall  f1-score   support

  strong_decrease     0.2635    0.2925    0.2772       318
moderate_decrease     0.1939    0.2194    0.2059       319
  slight_decrease     0.2080    0.3260    0.2540       319
             flat     0.1348    0.0757    0.0970       317
  slight_increase     0.3000    0.0564    0.0950       319
moderate_increase     0.1536    0.1478    0.1506       318
  strong_increase     0.2548    0.3762    0.3038       319

         accuracy                         0.2135      2229
        macro avg     0.2155    0.2134    0.1976      2229
     weighted avg     0.2156    0.2135    0.1977      2229

saved model: /content/drive/MyDrive/COMS Shared Data/finbert_cls_60min_7cls.pt
saved meta : /content/drive/MyDrive/COMS Shared Data/finbert_cls_60min_7cls_meta.parquet
all horizon/class models: {(1, 2): ('/content/drive/MyDrive/COMS Shared Data/finbert_cls_1min_2cls.pt', '/content/drive/MyDrive/COMS Shared Data/finber

In [ ]:
# ======================================================
# Cell NL0: manually specify best horizon for each class count
# (fill with your own choices from the previous experiments)
# ======================================================

BEST_HORIZON_FOR_CLASS = {
    2: 5,   # e.g. best test acc for 2-class was at 60min
    3: 1,   # e.g. best test acc for 3-class was at 30min
    7: 5,    # e.g. best test acc for 7-class was at 5min
}


In [ ]:
# ======================================================
# Cell NL1: build normalized log-return for needed horizons
# ======================================================
import os
import numpy as np
import pandas as pd
from math import pi

iv_path = "/content/drive/MyDrive/COMS Shared Data/ATM IV 30.xlsx"

def extract_ticker(col):
    if isinstance(col, str) and " " in col:
        return col.split()[0]
    return None

iv_raw = pd.read_excel(iv_path)
iv_raw = iv_raw[iv_raw.iloc[:, 0] != "DATES"].copy()

date_col = iv_raw.columns[0]
col_map = {col: extract_ticker(col) for col in iv_raw.columns}

value_cols = [col for col, tic in col_map.items() if tic is not None]
value_cols = [c for c in value_cols if c != date_col]
sel_cols = [date_col] + value_cols

iv_df = iv_raw[sel_cols].copy()
iv_df[date_col] = pd.to_datetime(iv_df[date_col])

for c in iv_df.columns:
    if c != date_col:
        iv_df[c] = pd.to_numeric(iv_df[c], errors="coerce")

iv_df[iv_df.columns[1:]] = iv_df[iv_df.columns[1:]].ffill()

rename_map = {c: col_map[c] for c in value_cols}
rename_map[date_col] = "DATES"
iv_df = iv_df.rename(columns=rename_map)

iv_df["DATES"] = pd.to_datetime(iv_df["DATES"]).dt.normalize()

iv_long = iv_df.melt(id_vars="DATES", var_name="ticker", value_name="iv")
iv_long = iv_long.sort_values(["ticker", "DATES"])
iv_long["iv_prev"] = iv_long.groupby("ticker", observed=True)["iv"].shift(1) / 100.0


def build_norm_log_for_horizon(horizon_minutes: int) -> str:
    news_path = os.path.join(BASE_DIR, f"news_with_ret_{horizon_minutes}min.parquet")
    df = pd.read_parquet(news_path)

    df["anchor_date"] = pd.to_datetime(df["time"]).dt.normalize()
    df["ticker"] = df["ticker"].astype(str)

    df = df.merge(
        iv_long[["DATES", "ticker", "iv_prev"]],
        left_on=["anchor_date", "ticker"],
        right_on=["DATES", "ticker"],
        how="left",
    ).drop(columns=["DATES"])

    ret_col = f"ret_after_news_{horizon_minutes}m"
    df = df.dropna(subset=[ret_col, "iv_prev"]).copy()
    df[ret_col] = df[ret_col].astype("float64")
    df["iv_prev"] = df["iv_prev"].astype("float64")

    T_days = 30.0
    t_minutes = float(horizon_minutes)
    t_over_T = t_minutes / (T_days * 24.0 * 60.0)

    k = np.sqrt(2.0 / pi) * np.sqrt(t_over_T)

    ret_abs = df[ret_col].abs()
    scale = df["iv_prev"] * k

    eps = 1e-12
    z_mag = np.log((ret_abs + eps) / (scale + eps))
    sign = np.sign(df[ret_col])

    df["log_ret_norm_signed"] = sign * z_mag

    norm_path = os.path.join(BASE_DIR, f"news_with_ret_{horizon_minutes}min_normlog.parquet")
    df.to_parquet(norm_path)
    print(
        f"H={horizon_minutes}min: saved normalized log-return ->",
        norm_path,
        "shape =", df.shape,
    )
    return norm_path


needed_horizons = sorted(set(BEST_HORIZON_FOR_CLASS.values()))
norm_paths = {h: build_norm_log_for_horizon(h) for h in needed_horizons}
print("normalized files:", norm_paths)
# Build normalized log-return for 30min and 60min as well
for h in [30, 60]:
    norm_path = os.path.join(BASE_DIR, f"news_with_ret_{h}min_normlog.parquet")
    if os.path.exists(norm_path):
        print(f"Skip horizon={h}min, file already exists:", norm_path)
    else:
        build_norm_log_for_horizon(h)


H=1min: saved normalized log-return -> /content/drive/MyDrive/COMS Shared Data/news_with_ret_1min_normlog.parquet shape = (22282, 19)
H=5min: saved normalized log-return -> /content/drive/MyDrive/COMS Shared Data/news_with_ret_5min_normlog.parquet shape = (22282, 19)
normalized files: {1: '/content/drive/MyDrive/COMS Shared Data/news_with_ret_1min_normlog.parquet', 5: '/content/drive/MyDrive/COMS Shared Data/news_with_ret_5min_normlog.parquet'}
Skip horizon=30min, file already exists: /content/drive/MyDrive/COMS Shared Data/news_with_ret_30min_normlog.parquet
Skip horizon=60min, file already exists: /content/drive/MyDrive/COMS Shared Data/news_with_ret_60min_normlog.parquet


In [ ]:
# ======================================================
# Cell: loop over 2 / 3 / 7 classes end-to-end
# ======================================================
import os
import re
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

def clean_summary_for_cls(s: str) -> str:
    s = str(s)
    s = re.sub(r"<[^>]+>", " ", s)
    s = re.sub(r"\s+", " ", s)
    if len(s) > 2000:
        s = s[:2000]
    return s.strip()

CLASS_CONFIGS = {
    7: {
        "label_col": "label_7cls",
        "n_bins": 7,
        "strategy": "quantile",
        "label_names": [
            "very_strong_down",
            "strong_down",
            "moderate_down",
            "flat",
            "moderate_up",
            "strong_up",
            "very_strong_up",
        ],
    },
    3: {
        "label_col": "label_3cls",
        "n_bins": 3,
        "strategy": "quantile",
        "label_names": [
            "down",
            "flat",
            "up",
        ],
    },
    2: {
        "label_col": "label_2cls",
        "n_bins": 2,
        "strategy": "sign",
        "label_names": [
            "down",
            "up",
        ],
    },
}

BATCH_SIZE_TRAIN = 16
BATCH_SIZE_TEST = 32
EPOCHS = 5
LR = 2e-5

results = {}

for num_classes, cfg in CLASS_CONFIGS.items():
    print("\n" + "=" * 80)
    print(f"Starting {num_classes}-class task")

    horizon = BEST_HORIZON_FOR_CLASS[num_classes]
    norm_path = os.path.join(
        BASE_DIR,
        f"news_with_ret_{horizon}min_normlog.parquet",
    )

    if not os.path.exists(norm_path):
        print(f"Normalized file for H={horizon}min not found, building it...")
        norm_path = build_norm_log_for_horizon(horizon)

    df = pd.read_parquet(norm_path)

    df = df.dropna(subset=["log_ret_norm_signed", "summary"]).copy()
    df["summary_clean_cls"] = df["summary"].apply(clean_summary_for_cls)
    df["text"] = df["ticker"].astype(str) + " [SEP] " + df["summary_clean_cls"]

    if cfg["strategy"] == "quantile":
        q_low, q_high = df["log_ret_norm_signed"].quantile([0.01, 0.99])
        df = df[df["log_ret_norm_signed"].between(q_low, q_high)].copy()
        df[cfg["label_col"]] = pd.qcut(
            df["log_ret_norm_signed"],
            q=cfg["n_bins"],
            labels=False,
            duplicates="drop",
        ).astype(int)
    elif cfg["strategy"] == "sign":
        df = df[df["log_ret_norm_signed"] != 0.0].copy()
        df[cfg["label_col"]] = (df["log_ret_norm_signed"] > 0.0).astype(int)
    else:
        raise ValueError(f"Unknown strategy {cfg['strategy']}")

    train_df, temp_df = train_test_split(
        df,
        test_size=0.2,
        random_state=42,
        stratify=df[cfg["label_col"]],
    )
    val_df, test_df = train_test_split(
        temp_df,
        test_size=0.5,
        random_state=43,
        stratify=temp_df[cfg["label_col"]],
    )

    print(
        f"{num_classes}-class normalized splits | "
        f"train / val / test: {len(train_df)}, {len(val_df)}, {len(test_df)}"
    )

    train_ds = ReturnDataset(train_df, text_col="text", label_col=cfg["label_col"])
    val_ds   = ReturnDataset(val_df,   text_col="text", label_col=cfg["label_col"])
    test_ds  = ReturnDataset(test_df,  text_col="text", label_col=cfg["label_col"])

    train_loader = DataLoader(
        train_ds,
        batch_size=BATCH_SIZE_TRAIN,
        shuffle=True,
        collate_fn=cls_collate_fn,
    )
    val_loader = DataLoader(
        val_ds,
        batch_size=BATCH_SIZE_TEST,
        shuffle=False,
        collate_fn=cls_collate_fn,
    )
    test_loader = DataLoader(
        test_ds,
        batch_size=BATCH_SIZE_TEST,
        shuffle=False,
        collate_fn=cls_collate_fn,
    )

    model = ReturnClassifier(num_labels=num_classes)
    model = train_classifier(
        model,
        train_loader,
        val_loader,
        num_epochs=EPOCHS,
        lr=LR,
        label_names=cfg["label_names"],
    )

    print(f"=== {num_classes}-class test performance ===")
    test_loss, test_acc = evaluate_classifier(
        model,
        test_loader,
        label_names=cfg["label_names"],
    )

    results[num_classes] = {
        "horizon": horizon,
        "test_loss": test_loss,
        "test_acc": test_acc,
    }

print("\n" + "=" * 80)
print("Summary of all tasks:")
for num_classes in sorted(results.keys()):
    r = results[num_classes]
    print(
        f"{num_classes}-class | "
        f"H={r['horizon']}min | "
        f"test_loss={r['test_loss']:.4f} | "
        f"test_acc={r['test_acc']:.4f}"
    )



Starting 7-class task
7-class normalized splits | train / val / test: 17474, 2184, 2185


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ProsusAI/finbert and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([7, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([7]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Train epoch 1/5:   0%|          | 0/1093 [00:00<?, ?it/s]

[Epoch 1] train_loss=1.9557, train_acc=0.1479


Eval:   0%|          | 0/69 [00:00<?, ?it/s]

...

[Epoch 5] train_loss=0.3448, train_acc=0.8497


Eval:   0%|          | 0/70 [00:00<?, ?it/s]

Eval loss=1.0477, acc=0.5467
              precision    recall  f1-score   support

        down     0.5537    0.5299    0.5416      1119
          up     0.5402    0.5639    0.5518      1096

    accuracy                         0.5467      2215
   macro avg     0.5469    0.5469    0.5467      2215
weighted avg     0.5470    0.5467    0.5466      2215

Best val acc = 0.5467
=== 2-class test performance ===


Eval:   0%|          | 0/70 [00:00<?, ?it/s]

Eval loss=1.0933, acc=0.5138
              precision    recall  f1-score   support

        down     0.5200    0.4991    0.5093      1120
          up     0.5079    0.5288    0.5181      1095

    accuracy                         0.5138      2215
   macro avg     0.5139    0.5139    0.5137      2215
weighted avg     0.5140    0.5138    0.5137      2215


Summary of all tasks:
2-class | H=5min | test_loss=1.0933 | test_acc=0.5138
3-class | H=1min | test_loss=1.7045 | test_acc=0.3626
7-class | H=5min | test_loss=2.4887 | test_acc=0.1890


In [ ]:
# ======================================================
# Volatility normalization experiments (window x bins) + summary
# ======================================================

import os
import re
import numpy as np
import pandas as pd
import torch
import torch.nn as nn

from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from tqdm.auto import tqdm
from peft import LoraConfig, get_peft_model, TaskType

# ---------------- Basic config ----------------
device = "cuda" if torch.cuda.is_available() else "cpu"
encoder_name = "ProsusAI/finbert"
tokenizer = AutoTokenizer.from_pretrained(encoder_name)

try:
    BASE_DIR
except NameError:
    BASE_DIR = "/content/drive/MyDrive/COMS Shared Data"

RET_1MIN_PATH = os.path.join(BASE_DIR, "ret_df_1min.parquet")
ATM_IV_PATH = os.path.join(BASE_DIR, "ATM IV 30.xlsx")

MAX_LEN = 256
BATCH_SIZE_TRAIN = 16
BATCH_SIZE_TEST  = 32
EPOCHS = 5

LABEL_MAP = {
    2: ["low_vol", "high_vol"],
    3: ["low_vol", "mid_vol", "high_vol"],
    7: [
        "very_low",
        "low",
        "slightly_low",
        "medium",
        "slightly_high",
        "high",
        "very_high",
    ],
}

def clean_text(s):
    s = str(s)
    s = re.sub(r"<[^>]+>", " ", s)
    s = re.sub(r"\s+", " ", s)
    if len(s) > 2000:
        s = s[:2000]
    return s.strip()

def build_input(row):
    src = row.get("source", "")
    if pd.isna(src):
        src = ""
    return f"[TICKER] {row['ticker']} [SRC] {src} [TEXT] {row['summary_clean']}"

# ---------------- Read iv_df from ATM IV Excel ----------------

def extract_ticker(col):
    if isinstance(col, str) and " " in col:
        return col.split()[0]
    return None

iv_raw = pd.read_excel(ATM_IV_PATH)
iv_raw = iv_raw[iv_raw.iloc[:, 0] != "DATES"].copy()

date_col = iv_raw.columns[0]
col_map = {col: extract_ticker(col) for col in iv_raw.columns}

value_cols = [col for col, tic in col_map.items() if tic is not None]
value_cols = [c for c in value_cols if c != date_col]
sel_cols = [date_col] + value_cols

iv_df = iv_raw[sel_cols].copy()
iv_df[date_col] = pd.to_datetime(iv_df[date_col])

for c in iv_df.columns:
    if c != date_col:
        iv_df[c] = pd.to_numeric(iv_df[c], errors="coerce")

iv_df[iv_df.columns[1:]] = iv_df[iv_df.columns[1:]].ffill()

rename_map = {c: col_map[c] for c in value_cols}
rename_map[date_col] = "DATES"
iv_df = iv_df.rename(columns=rename_map)

iv_df["DATES"] = pd.to_datetime(iv_df["DATES"]).dt.normalize()

iv_long = iv_df.melt(id_vars="DATES", var_name="ticker", value_name="iv")
iv_long = iv_long.sort_values(["ticker", "DATES"])
iv_long["iv_prev"] = iv_long.groupby("ticker", observed=True)["iv"].shift(1) / 100.0

# ---------------- LoRA FinBERT classifier ----------------

class NewsVolDataset(Dataset):
    def __init__(self, df, tokenizer, max_len=256):
        self.texts = df["input_text"].tolist()
        self.labels = df["label"].values.astype("int64")
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        enc = self.tokenizer(
            self.texts[idx],
            truncation=True,
            padding="max_length",
            max_length=self.max_len,
            return_tensors="pt",
        )
        item = {k: v.squeeze(0) for k, v in enc.items()}
        item["labels"] = self.labels[idx]
        return item

class FinBertClassifierLoRA(nn.Module):
    def __init__(self, encoder_name, num_labels=2):
        super().__init__()
        self.encoder = AutoModel.from_pretrained(encoder_name)
        peft_config = LoraConfig(
            task_type=TaskType.FEATURE_EXTRACTION,
            inference_mode=False,
            r=8,
            lora_alpha=32,
            lora_dropout=0.1,
            target_modules=["query", "value"],
        )
        self.encoder = get_peft_model(self.encoder, peft_config)
        hidden_size = self.encoder.config.hidden_size
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(hidden_size, num_labels)

    def forward(self, input_ids, attention_mask, labels=None):
        out = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
        cls_hidden = out.last_hidden_state[:, 0, :]
        logits = self.classifier(self.dropout(cls_hidden))
        loss = None
        if labels is not None:
            loss = nn.CrossEntropyLoss()(logits, labels)
        return {"loss": loss, "logits": logits}

# ---------------- Load 1-min returns ----------------

ret_df = pd.read_parquet(RET_1MIN_PATH)
ret_df["time"] = pd.to_datetime(ret_df["time"])
ret_df = ret_df.sort_values(["ticker", "time"]).reset_index(drop=True)
ret_df["date"] = ret_df["time"].dt.date
ret_df["log_ret"] = np.log1p(ret_df["ret_1m"])

WINDOWS = [5, 30, 60]
BINS = [2, 3, 7]

results_vol = {}

for window in WINDOWS:
    print("\n" + "=" * 80)
    print(f"Window = {window} minutes")

    df = ret_df.copy()
    g = df.groupby(["ticker", "date"], observed=True)

    df["vol_tmp"] = g["log_ret"].rolling(window).std().reset_index(level=[0, 1], drop=True)
    vol_col = f"vol_future_{window}m"
    df[vol_col] = g["vol_tmp"].shift(-window)
    df = df.drop(columns=["vol_tmp"])

    vol_df = df.dropna(subset=[vol_col])[["ticker", "time", vol_col]].copy()

    available = vol_df[["ticker", "time"]].drop_duplicates().sort_values(["ticker", "time"])
    ticker2times = {}
    for t, sub in available.groupby("ticker", observed=True):
        ticker2times[t] = pd.DatetimeIndex(sub["time"])

    def compute_anchor(row):
        t = row["ticker"]
        ts = row["news_time"]
        tf = row["time_floor"]
        times = ticker2times.get(t)
        if times is None or len(times) == 0:
            return pd.NaT
        if tf in times:
            return tf
        idx = times.searchsorted(ts)
        if idx >= len(times):
            return pd.NaT
        return times[idx]

    news_df = news_filtered.copy()
    news_df["news_time"] = pd.to_datetime(news_df["time"]).dt.tz_localize(None)
    news_df["time_floor"] = news_df["news_time"].dt.floor("min")
    news_df["anchor_time"] = news_df.apply(compute_anchor, axis=1)

    anchors = news_df[news_df["anchor_time"].notna()].copy()
    merged_df = anchors.merge(
        vol_df,
        left_on=["ticker", "anchor_time"],
        right_on=["ticker", "time"],
        how="inner",
    )

    merged_df = merged_df.dropna(subset=[vol_col])
    merged_df = merged_df.sort_values(["anchor_time", "ticker"]).reset_index(drop=True)

    merged_df["anchor_date"] = pd.to_datetime(merged_df["anchor_time"]).dt.normalize()
    merged_df = merged_df.merge(
        iv_long[["DATES", "ticker", "iv_prev"]],
        left_on=["anchor_date", "ticker"],
        right_on=["DATES", "ticker"],
        how="left",
    ).drop(columns=["DATES"])

    merged_df["vol_norm"] = merged_df[vol_col] / merged_df["iv_prev"]
    merged_df = merged_df.dropna(subset=["vol_norm"])

    print(f"Available samples after vol and IV merge: {len(merged_df)}")

    for num_bins in BINS:
        print("-" * 60)
        print(f"Bins = {num_bins}")
        label_names = LABEL_MAP[num_bins]

        df_bin = merged_df.copy()

        if num_bins == 2:
            q_low = df_bin["vol_norm"].quantile(0.3)
            q_high = df_bin["vol_norm"].quantile(0.7)
            labels = np.full(len(df_bin), -1, dtype=int)
            labels[df_bin["vol_norm"] <= q_low] = 0
            labels[df_bin["vol_norm"] >= q_high] = 1
        else:
            labels = pd.qcut(
                df_bin["vol_norm"],
                q=num_bins,
                labels=False,
                duplicates="drop",
            ).astype(int)

        df_bin["label"] = labels
        df_bin = df_bin[df_bin["label"] != -1].copy()

        if df_bin["label"].nunique() < num_bins:
            print(
                f"Warning: only {df_bin['label'].nunique()} unique labels "
                f"for num_bins={num_bins}, skipping."
            )
            continue

        df_bin["summary_clean"] = df_bin["summary"].apply(clean_text)
        df_bin["input_text"] = df_bin.apply(build_input, axis=1)

        train_df, test_df = train_test_split(
            df_bin,
            test_size=0.2,
            random_state=42,
            stratify=df_bin["label"],
        )

        train_ds = NewsVolDataset(train_df, tokenizer, max_len=MAX_LEN)
        test_ds = NewsVolDataset(test_df, tokenizer, max_len=MAX_LEN)

        train_loader = DataLoader(
            train_ds,
            batch_size=BATCH_SIZE_TRAIN,
            shuffle=True,
        )
        test_loader = DataLoader(
            test_ds,
            batch_size=BATCH_SIZE_TEST,
            shuffle=False,
        )

        model = FinBertClassifierLoRA(encoder_name, num_labels=num_bins).to(device)
        optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

        for epoch in range(EPOCHS):
            model.train()
            total_loss = 0.0
            for batch in tqdm(
                train_loader,
                desc=f"Train epoch {epoch+1} (win={window}, bins={num_bins})",
                leave=False,
            ):
                input_ids = batch["input_ids"].to(device)
                attention_mask = batch["attention_mask"].to(device)
                labels_tensor = batch["labels"].to(device)

                optimizer.zero_grad()
                out = model(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    labels=labels_tensor,
                )
                loss = out["loss"]
                loss.backward()
                optimizer.step()
                total_loss += loss.item()

            avg_loss = total_loss / max(1, len(train_loader))
            print(
                f"[window={window}, bins={num_bins}] "
                f"Epoch {epoch+1} train_loss = {avg_loss:.4f}"
            )

        model.eval()
        all_labels, all_preds = [], []

        with torch.no_grad():
            for batch in tqdm(
                test_loader,
                desc=f"Evaluating (win={window}, bins={num_bins})",
                leave=False,
            ):
                input_ids = batch["input_ids"].to(device)
                attention_mask = batch["attention_mask"].to(device)
                labels_np = batch["labels"].cpu().numpy()

                logits = model(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                )["logits"]
                preds_np = logits.argmax(dim=-1).cpu().numpy()

                all_labels.append(labels_np)
                all_preds.append(preds_np)

        all_labels = np.concatenate(all_labels)
        all_preds = np.concatenate(all_preds)

        acc = accuracy_score(all_labels, all_preds)
        print(
            f"[window={window}, bins={num_bins}] "
            f"Test accuracy: {acc:.4f}"
        )
        print(classification_report(all_labels, all_preds, target_names=label_names))

        results_vol[(window, num_bins)] = {
            "n_samples": len(df_bin),
            "test_acc": acc,
        }

print("\n" + "=" * 80)
print("Summary of volatility normalization experiments:")
for (window, num_bins), res in sorted(results_vol.items()):
    print(
        f"window={window:>2} min, bins={num_bins}: "
        f"n={res['n_samples']}, test_acc={res['test_acc']:.4f}"
    )

print("\n" + "=" * 80)
print("Best configuration per number of bins:")

best_per_bins = {}
for num_bins in BINS:
    best_key = None
    best_val = None
    for (window, b), res in results_vol.items():
        if b != num_bins:
            continue
        if best_val is None or res["test_acc"] > best_val["test_acc"]:
            best_key = (window, b)
            best_val = res
    if best_val is not None:
        best_per_bins[num_bins] = {
            "window": best_key[0],
            "n_samples": best_val["n_samples"],
            "test_acc": best_val["test_acc"],
        }
        print(
            f"bins={num_bins}: "
            f"best_window={best_key[0]} min, "
            f"n={best_val['n_samples']}, "
            f"test_acc={best_val['test_acc']:.4f}"
        )
    else:
        print(f"bins={num_bins}: no valid results.")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
# ======================================================
# Cell 1: FinBERT head-only baselines
# ======================================================

import os
import re
import numpy as np
import pandas as pd
import torch
import torch.nn as nn

from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from tqdm.auto import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"
encoder_name = "ProsusAI/finbert"
tokenizer = AutoTokenizer.from_pretrained(encoder_name)

try:
    BASE_DIR
except NameError:
    BASE_DIR = "/content/drive/MyDrive/COMS Shared Data"

RET_1MIN_PATH = os.path.join(BASE_DIR, "ret_df_1min.parquet")
ATM_IV_PATH = os.path.join(BASE_DIR, "ATM IV 30.xlsx")

MAX_LEN = 256
BATCH_SIZE_TRAIN = 16
BATCH_SIZE_TEST = 32
EPOCHS = 5
LR_HEAD = 1e-4

RET_LABEL_MAP = {
    2: ["neg", "pos"],
    3: ["down", "flat", "up"],
    7: [
        "very_down",
        "down",
        "slightly_down",
        "flat",
        "slightly_up",
        "up",
        "very_up",
    ],
}

VOL_LABEL_MAP = {
    2: ["low_vol", "high_vol"],
    3: ["low_vol", "mid_vol", "high_vol"],
    7: [
        "very_low",
        "low",
        "slightly_low",
        "medium",
        "slightly_high",
        "high",
        "very_high",
    ],
}


def clean_text(s):
    s = str(s)
    s = re.sub(r"<[^>]+>", " ", s)
    s = re.sub(r"\s+", " ", s)
    if len(s) > 2000:
        s = s[:2000]
    return s.strip()


class TextDataset(Dataset):
    def __init__(self, df, text_col, label_col):
        self.texts = df[text_col].tolist()
        self.labels = df[label_col].astype("int64").tolist()

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return {"text": self.texts[idx], "label": self.labels[idx]}


def cls_collate_fn(batch):
    texts = [b["text"] for b in batch]
    labels = torch.tensor([b["label"] for b in batch], dtype=torch.long)
    enc = tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=MAX_LEN,
        return_tensors="pt",
    )
    enc["labels"] = labels
    return enc


class HeadOnlyFinBert(nn.Module):
    def __init__(self, encoder_name, num_labels: int):
        super().__init__()
        self.encoder = AutoModel.from_pretrained(encoder_name)
        hidden_size = self.encoder.config.hidden_size
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(hidden_size, num_labels)

        for param in self.encoder.parameters():
            param.requires_grad = False

    def forward(self, input_ids=None, attention_mask=None, labels=None):
        out = self.encoder(
            input_ids=input_ids,
            attention_mask=attention_mask,
        )
        cls_hidden = out.last_hidden_state[:, 0, :]
        logits = self.classifier(self.dropout(cls_hidden))
        loss = None
        if labels is not None:
            loss = nn.CrossEntropyLoss()(logits, labels)
        return {"loss": loss, "logits": logits}


def evaluate_model(model, data_loader, label_names=None):
    model.eval()
    total_loss = 0.0
    total_correct = 0
    total_examples = 0
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for batch in tqdm(data_loader, desc="Eval", leave=False):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels,
            )
            loss = outputs["loss"]
            logits = outputs["logits"]

            total_loss += loss.item() * labels.size(0)
            preds = logits.argmax(dim=-1)
            total_correct += (preds == labels).sum().item()
            total_examples += labels.size(0)

            all_labels.extend(labels.cpu().numpy().tolist())
            all_preds.extend(preds.cpu().numpy().tolist())

    avg_loss = total_loss / total_examples if total_examples > 0 else 0.0
    acc = total_correct / total_examples if total_examples > 0 else 0.0
    print(f"loss={avg_loss:.4f}, acc={acc:.4f}")
    if label_names is not None:
        print(
            classification_report(
                all_labels, all_preds, target_names=label_names, digits=4
            )
        )
    return avg_loss, acc


def train_headonly_model(
    train_loader,
    val_loader,
    num_labels: int,
    epochs: int = EPOCHS,
    lr: float = LR_HEAD,
    label_names=None,
):
    model = HeadOnlyFinBert(encoder_name, num_labels=num_labels)
    model.to(device)
    optimizer = torch.optim.AdamW(
        [p for p in model.parameters() if p.requires_grad],
        lr=lr,
    )

    best_state = None
    best_val_acc = 0.0

    for epoch in range(epochs):
        model.train()
        total_loss = 0.0
        total_correct = 0
        total_examples = 0

        for batch in tqdm(
            train_loader,
            desc=f"Train epoch {epoch+1}/{epochs}",
            leave=False,
        ):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            optimizer.zero_grad()
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels,
            )
            loss = outputs["loss"]
            logits = outputs["logits"]

            loss.backward()
            optimizer.step()

            total_loss += loss.item() * labels.size(0)
            preds = logits.argmax(dim=-1)
            total_correct += (preds == labels).sum().item()
            total_examples += labels.size(0)

        train_loss = total_loss / total_examples
        train_acc = total_correct / total_examples
        print(f"[Epoch {epoch+1}] train_loss={train_loss:.4f}, train_acc={train_acc:.4f}")

        print("Validation:")
        _, val_acc = evaluate_model(model, val_loader, label_names=label_names)

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_state = model.state_dict()

    if best_state is not None:
        model.load_state_dict(best_state)
    print(f"Best val acc={best_val_acc:.4f}")
    return model


# ---------- normalized log-return dataset builder ----------

def prepare_norm_log_dataset(horizon_minutes: int, num_bins: int):
    assert num_bins in {2, 3, 7}
    path = os.path.join(BASE_DIR, f"news_with_ret_{horizon_minutes}min_normlog.parquet")
    df = pd.read_parquet(path)

    df = df.dropna(subset=["log_ret_norm_signed", "summary"]).copy()
    q_low, q_high = df["log_ret_norm_signed"].quantile([0.01, 0.99])
    df = df[df["log_ret_norm_signed"].between(q_low, q_high)].copy()

    x = df["log_ret_norm_signed"].values
    if num_bins == 2:
        p_low, p_high = np.quantile(x, [0.3, 0.7])
        labels = np.full(len(df), -1, dtype=int)
        labels[x <= p_low] = 0
        labels[x >= p_high] = 1
    else:
        labels = pd.qcut(
            df["log_ret_norm_signed"],
            q=num_bins,
            labels=False,
            duplicates="drop",
        ).astype(int).values

    df = df.copy()
    df["label_cls"] = labels
    df = df[df["label_cls"] != -1].copy()
    df["label_cls"] = df["label_cls"].astype(int)

    df["summary_clean"] = df["summary"].apply(clean_text)
    df["text"] = df["ticker"].astype(str) + " [SEP] " + df["summary_clean"]

    label_names = RET_LABEL_MAP[num_bins]

    train_df, temp_df = train_test_split(
        df,
        test_size=0.2,
        random_state=42,
        stratify=df["label_cls"],
    )
    val_df, test_df = train_test_split(
        temp_df,
        test_size=0.5,
        random_state=43,
        stratify=temp_df["label_cls"],
    )

    return train_df, val_df, test_df, label_names


# ---------- normalized volatility dataset builder ----------

def extract_ticker(col):
    if isinstance(col, str) and " " in col:
        return col.split()[0]
    return None


iv_raw = pd.read_excel(ATM_IV_PATH)
iv_raw = iv_raw[iv_raw.iloc[:, 0] != "DATES"].copy()
date_col_iv = iv_raw.columns[0]
col_map_iv = {col: extract_ticker(col) for col in iv_raw.columns}
value_cols_iv = [col for col, tic in col_map_iv.items() if tic is not None and col != date_col_iv]
sel_cols_iv = [date_col_iv] + value_cols_iv

iv_df = iv_raw[sel_cols_iv].copy()
iv_df[date_col_iv] = pd.to_datetime(iv_df[date_col_iv])
for c in iv_df.columns:
    if c != date_col_iv:
        iv_df[c] = pd.to_numeric(iv_df[c], errors="coerce")
iv_df[iv_df.columns[1:]] = iv_df[iv_df.columns[1:]].ffill()
rename_iv = {c: col_map_iv[c] for c in value_cols_iv}
rename_iv[date_col_iv] = "DATES"
iv_df = iv_df.rename(columns=rename_iv)
iv_df["DATES"] = pd.to_datetime(iv_df["DATES"]).dt.normalize()

iv_long = iv_df.melt(id_vars="DATES", var_name="ticker", value_name="iv")
iv_long = iv_long.sort_values(["ticker", "DATES"])
iv_long["iv_prev"] = iv_long.groupby("ticker", observed=True)["iv"].shift(1) / 100.0

ret_df_base = pd.read_parquet(RET_1MIN_PATH)
ret_df_base["time"] = pd.to_datetime(ret_df_base["time"])
ret_df_base = ret_df_base.sort_values(["ticker", "time"]).reset_index(drop=True)
ret_df_base["date"] = ret_df_base["time"].dt.date
ret_df_base["log_ret"] = np.log1p(ret_df_base["ret_1m"])
g_base = ret_df_base.groupby(["ticker", "date"], observed=True)
for w in [5, 30, 60]:
    tmp_col = f"vol_tmp_{w}"
    vol_col = f"vol_future_{w}m"
    ret_df_base[tmp_col] = g_base["log_ret"].rolling(w).std().reset_index(level=[0, 1], drop=True)
    ret_df_base[vol_col] = g_base[tmp_col].shift(-w)
    ret_df_base.drop(columns=[tmp_col], inplace=True)

VOL_BASE = {}


def get_vol_norm_base(window: int):
    if window in VOL_BASE:
        return VOL_BASE[window].copy()

    vol_col = f"vol_future_{window}m"
    vol_df = ret_df_base.dropna(subset=[vol_col])[["ticker", "time", vol_col]].copy()

    available = vol_df[["ticker", "time"]].drop_duplicates().sort_values(["ticker", "time"])
    ticker2times = {}
    for t, sub in available.groupby("ticker", observed=True):
        ticker2times[t] = pd.DatetimeIndex(sub["time"])

    news_df = news_filtered.copy()
    news_df["news_time"] = pd.to_datetime(news_df["time"]).dt.tz_localize(None)
    news_df["time_floor"] = news_df["news_time"].dt.floor("min")

    def compute_anchor(row):
        t = row["ticker"]
        ts = row["news_time"]
        tf = row["time_floor"]
        times = ticker2times.get(t)
        if times is None or len(times) == 0:
            return pd.NaT
        if tf in times:
            return tf
        idx = times.searchsorted(ts)
        if idx >= len(times):
            return pd.NaT
        return times[idx]

    news_df["anchor_time"] = news_df.apply(compute_anchor, axis=1)
    anchors = news_df[news_df["anchor_time"].notna()].copy()

    merged_df = anchors.merge(
        vol_df,
        left_on=["ticker", "anchor_time"],
        right_on=["ticker", "time"],
        how="inner",
    )
    merged_df = merged_df.dropna(subset=[vol_col])
    merged_df = merged_df.sort_values(["anchor_time", "ticker"]).reset_index(drop=True)

    merged_df["anchor_date"] = pd.to_datetime(merged_df["anchor_time"]).dt.normalize()
    merged_df = merged_df.merge(
        iv_long[["DATES", "ticker", "iv_prev"]],
        left_on=["anchor_date", "ticker"],
        right_on=["DATES", "ticker"],
        how="left",
    ).drop(columns=["DATES"])

    merged_df["vol_norm"] = merged_df[vol_col] / merged_df["iv_prev"]
    merged_df = merged_df.dropna(subset=["vol_norm"])
    merged_df["summary_clean"] = merged_df["summary"].apply(clean_text)
    merged_df["text"] = merged_df.apply(
        lambda r: f"[TICKER] {r['ticker']} [SRC] {r.get('source', '')} [TEXT] {r['summary_clean']}",
        axis=1,
    )

    VOL_BASE[window] = merged_df
    return merged_df.copy()


def prepare_vol_norm_dataset(window: int, num_bins: int):
    assert num_bins in {2, 3, 7}
    df = get_vol_norm_base(window)
    x = df["vol_norm"].values

    if num_bins == 2:
        p_low, p_high = np.quantile(x, [0.3, 0.7])
        labels = np.full(len(df), -1, dtype=int)
        labels[x <= p_low] = 0
        labels[x >= p_high] = 1
    else:
        labels = pd.qcut(
            df["vol_norm"],
            q=num_bins,
            labels=False,
            duplicates="drop",
        ).astype(int).values

    df = df.copy()
    df["label_cls"] = labels
    df = df[df["label_cls"] != -1].copy()
    df["label_cls"] = df["label_cls"].astype(int)

    label_names = VOL_LABEL_MAP[num_bins]

    train_df, temp_df = train_test_split(
        df,
        test_size=0.2,
        random_state=42,
        stratify=df["label_cls"],
    )
    val_df, test_df = train_test_split(
        temp_df,
        test_size=0.5,
        random_state=43,
        stratify=temp_df["label_cls"],
    )

    return train_df, val_df, test_df, label_names


RET_HORIZONS = [1, 5, 30, 60]
RET_BINS = [2, 3, 7]
VOL_WINDOWS = [5, 30, 60]
VOL_BINS = [2, 3, 7]

results_finbert = {}

print("\n=== Normalized log-return tasks ===")
for h in RET_HORIZONS:
    for b in RET_BINS:
        print("\n" + "-" * 80)
        print(f"Log-return: horizon={h}min, bins={b}")
        train_df, val_df, test_df, label_names = prepare_norm_log_dataset(h, b)

        train_ds = TextDataset(train_df, text_col="text", label_col="label_cls")
        val_ds = TextDataset(val_df, text_col="text", label_col="label_cls")
        test_ds = TextDataset(test_df, text_col="text", label_col="label_cls")

        train_loader = DataLoader(
            train_ds,
            batch_size=BATCH_SIZE_TRAIN,
            shuffle=True,
            collate_fn=cls_collate_fn,
        )
        val_loader = DataLoader(
            val_ds,
            batch_size=BATCH_SIZE_TEST,
            shuffle=False,
            collate_fn=cls_collate_fn,
        )
        test_loader = DataLoader(
            test_ds,
            batch_size=BATCH_SIZE_TEST,
            shuffle=False,
            collate_fn=cls_collate_fn,
        )

        model = train_headonly_model(
            train_loader,
            val_loader,
            num_labels=b,
            epochs=EPOCHS,
            lr=LR_HEAD,
            label_names=label_names,
        )
        print("Test:")
        _, test_acc = evaluate_model(model, test_loader, label_names=label_names)
        results_finbert[("logret", h, b)] = {
            "n_train": len(train_df),
            "n_test": len(test_df),
            "test_acc": test_acc,
        }

print("\n=== Normalized volatility tasks ===")
for w in VOL_WINDOWS:
    for b in VOL_BINS:
        print("\n" + "-" * 80)
        print(f"Volatility: window={w}min, bins={b}")
        train_df, val_df, test_df, label_names = prepare_vol_norm_dataset(w, b)

        train_ds = TextDataset(train_df, text_col="text", label_col="label_cls")
        val_ds = TextDataset(val_df, text_col="text", label_col="label_cls")
        test_ds = TextDataset(test_df, text_col="text", label_col="label_cls")

        train_loader = DataLoader(
            train_ds,
            batch_size=BATCH_SIZE_TRAIN,
            shuffle=True,
            collate_fn=cls_collate_fn,
        )
        val_loader = DataLoader(
            val_ds,
            batch_size=BATCH_SIZE_TEST,
            shuffle=False,
            collate_fn=cls_collate_fn,
        )
        test_loader = DataLoader(
            test_ds,
            batch_size=BATCH_SIZE_TEST,
            shuffle=False,
            collate_fn=cls_collate_fn,
        )

        model = train_headonly_model(
            train_loader,
            val_loader,
            num_labels=b,
            epochs=EPOCHS,
            lr=LR_HEAD,
            label_names=label_names,
        )
        print("Test:")
        _, test_acc = evaluate_model(model, test_loader, label_names=label_names)
        results_finbert[("vol", w, b)] = {
            "n_train": len(train_df),
            "n_test": len(test_df),
            "test_acc": test_acc,
        }

print("\n=== Summary: FinBERT head-only baselines ===")
for key, res in sorted(results_finbert.items()):
    kind, horizon, bins = key
    print(
        f"{kind} | h={horizon:>2} min, bins={bins}: "
        f"n_train={res['n_train']}, n_test={res['n_test']}, "
        f"test_acc={res['test_acc']:.4f}"
    )



=== Normalized log-return tasks ===

--------------------------------------------------------------------------------
Log-return: horizon=1min, bins=2


Train epoch 1/5:   0%|          | 0/656 [00:00<?, ?it/s]

[Epoch 1] train_loss=0.7058, train_acc=0.5058
Validation:


Eval:   0%|          | 0/41 [00:00<?, ?it/s]

loss=0.6985, acc=0.5038
              precision    recall  f1-score   support

         neg     0.5037    0.5130    0.5083       655
         pos     0.5039    0.4947    0.4992       655

    accuracy                         0.5038      1310
   macro avg     0.5038    0.5038    0.5038      1310
weighted avg     0.5038    0.5038    0.5038      1310



...

[Epoch 5] train_loss=1.9287, train_acc=0.1877
Validation:


Eval:   0%|          | 0/70 [00:00<?, ?it/s]

loss=1.9298, acc=0.1842
               precision    recall  f1-score   support

     very_low     0.1853    0.1661    0.1752       319
          low     0.1881    0.1289    0.1530       318
 slightly_low     0.1531    0.1003    0.1212       319
       medium     0.1111    0.0282    0.0450       319
slightly_high     0.1752    0.3448    0.2323       319
         high     0.1894    0.0786    0.1111       318
    very_high     0.2083    0.4420    0.2831       319

     accuracy                         0.1842      2231
    macro avg     0.1729    0.1842    0.1601      2231
 weighted avg     0.1729    0.1842    0.1602      2231

Best val acc=0.1842
Test:


Eval:   0%|          | 0/70 [00:00<?, ?it/s]

loss=1.9346, acc=0.1770
               precision    recall  f1-score   support

     very_low     0.1978    0.1724    0.1843       319
          low     0.1502    0.1097    0.1268       319
 slightly_low     0.1184    0.0909    0.1028       319
       medium     0.1184    0.0282    0.0456       319
slightly_high     0.1709    0.3386    0.2271       319
         high     0.1415    0.0472    0.0708       318
    very_high     0.2175    0.4514    0.2936       319

     accuracy                         0.1770      2232
    macro avg     0.1593    0.1769    0.1501      2232
 weighted avg     0.1593    0.1770    0.1502      2232


=== Summary: FinBERT head-only baselines ===
logret | h= 1 min, bins=2: n_train=10481, n_test=1311, test_acc=0.5126
logret | h= 1 min, bins=3: n_train=17468, n_test=2184, test_acc=0.3361
logret | h= 1 min, bins=7: n_train=17468, n_test=2184, test_acc=0.1589
logret | h= 5 min, bins=2: n_train=10486, n_test=1311, test_acc=0.4981
logret | h= 5 min, bins=3: n_train=174

In [ ]:
# ======================================================
# Cell: Zero-shot FinBERT baselines for log-return & volatility

# ======================================================

import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score, classification_report
from tqdm.auto import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"
encoder_name = "ProsusAI/finbert"
finbert_tokenizer = AutoTokenizer.from_pretrained(encoder_name)

# 加载 FinBERT 原始 sentiment 模型 (3 labels: neg, neu, pos)
finbert_sent = AutoModelForSequenceClassification.from_pretrained(
    encoder_name
).to(device)
finbert_sent.eval()


class FinbertEvalDataset(Dataset):
    def __init__(self, df, text_col: str, label_col: str):
        self.texts = df[text_col].tolist()
        self.labels = df[label_col].astype("int64").tolist()

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return {
            "text": self.texts[idx],
            "label": self.labels[idx],
        }


def finbert_collate_fn(batch):
    texts = [b["text"] for b in batch]
    labels = torch.tensor([b["label"] for b in batch], dtype=torch.long)
    enc = finbert_tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=256,
        return_tensors="pt",
    )
    enc["labels"] = labels
    return enc


@torch.no_grad()
def eval_finbert_zero_shot_quantile(df, num_bins: int, task_type: str, label_names):


    ds = FinbertEvalDataset(df, text_col="text", label_col="label_cls")
    loader = DataLoader(
        ds,
        batch_size=64,
        shuffle=False,
        collate_fn=finbert_collate_fn,
    )

    all_labels = []
    all_scores = []

    for batch in tqdm(loader, desc=f"FinBERT zero-shot ({task_type}, bins={num_bins})", leave=False):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].cpu().numpy()

        outputs = finbert_sent(
            input_ids=input_ids,
            attention_mask=attention_mask,
        )
        logits = outputs.logits  # [B,3]
        probs = torch.softmax(logits, dim=-1)  # (neg, neu, pos)

        p_neg = probs[:, 0]
        p_neu = probs[:, 1]
        p_pos = probs[:, 2]

        if task_type == "logret":
            # sentiment direction: positive vs negative
            score = (p_pos - p_neg).cpu().numpy()
        elif task_type == "vol":
            # sentiment intensity: 1 - p_neu
            score = (1.0 - p_neu).cpu().numpy()
        else:
            raise ValueError("task_type must be 'logret' or 'vol'.")

        all_labels.append(labels)
        all_scores.append(score)

    all_labels = np.concatenate(all_labels)
    all_scores = np.concatenate(all_scores)

    # 分位数分箱: 保证输出 label 在 [0, num_bins-1]
    qs = np.linspace(0.0, 1.0, num_bins + 1)
    edges = np.quantile(all_scores, qs)

    eps = 1e-8
    for i in range(1, len(edges)):
        if edges[i] <= edges[i - 1]:
            edges[i] = edges[i - 1] + eps

    preds = np.searchsorted(edges[1:-1], all_scores, side="right")

    acc = accuracy_score(all_labels, preds)
    print(f"Accuracy = {acc:.4f}")
    print(
        classification_report(
            all_labels,
            preds,
            target_names=label_names,
            digits=4,
        )
    )
    return acc



RET_HORIZONS = [1, 5, 30, 60]
RET_BINS = [2, 3, 7]

VOL_WINDOWS = [5, 30, 60]
VOL_BINS = [2, 3, 7]

finbert_zero_shot_results = {}

print("\n=== Zero-shot FinBERT baselines on normalized log-return ===")
for h in RET_HORIZONS:
    for b in RET_BINS:
        print("\n" + "-" * 80)
        print(f"[LOGRET] horizon={h}min, bins={b}")
        train_df, val_df, test_df, label_names = prepare_norm_log_dataset(h, b)
        acc = eval_finbert_zero_shot_quantile(
            df=test_df,
            num_bins=b,
            task_type="logret",
            label_names=label_names,
        )
        finbert_zero_shot_results[("logret", h, b)] = {
            "n_test": len(test_df),
            "test_acc": acc,
        }

print("\n=== Zero-shot FinBERT baselines on normalized volatility ===")
for w in VOL_WINDOWS:
    for b in VOL_BINS:
        print("\n" + "-" * 80)
        print(f"[VOL] window={w}min, bins={b}")
        train_df, val_df, test_df, label_names = prepare_vol_norm_dataset(w, b)
        acc = eval_finbert_zero_shot_quantile(
            df=test_df,
            num_bins=b,
            task_type="vol",
            label_names=label_names,
        )
        finbert_zero_shot_results[("vol", w, b)] = {
            "n_test": len(test_df),
            "test_acc": acc,
        }

print("\n=== Summary: zero-shot FinBERT baselines (log-return + vol) ===")
for (kind, horizon, bins), res in sorted(finbert_zero_shot_results.items()):
    tag = "h" if kind == "logret" else "w"
    print(
        f"{kind} | {tag}={horizon:>2} min, bins={bins}: "
        f"n_test={res['n_test']}, acc={res['test_acc']:.4f}"
    )



=== Zero-shot FinBERT baselines on normalized log-return ===

--------------------------------------------------------------------------------
[LOGRET] horizon=1min, bins=2


FinBERT zero-shot (logret, bins=2):   0%|          | 0/21 [00:00<?, ?it/s]

Accuracy = 0.5233
              precision    recall  f1-score   support

         neg     0.5237    0.5229    0.5233       656
         pos     0.5229    0.5237    0.5233       655

    accuracy                         0.5233      1311
   macro avg     0.5233    0.5233    0.5233      1311
weighted avg     0.5233    0.5233    0.5233      1311


--------------------------------------------------------------------------------
[LOGRET] horizon=1min, bins=3


FinBERT zero-shot (logret, bins=3):   0%|          | 0/35 [00:00<?, ?it/s]

Accuracy = 0.3297
              precision    recall  f1-score   support

        down     0.3187    0.3187    0.3187       728
        flat     0.3242    0.3242    0.3242       728
          up     0.3462    0.3462    0.3462       728

    accuracy                         0.3297      2184
   macro avg     0.3297    0.3297    0.3297      2184
weighted avg     0.3297    0.3297    0.3297      2184


--------------------------------------------------------------------------------
[LOGRET] horizon=1min, bins=7


...

Accuracy = 0.5254
              precision    recall  f1-score   support

     low_vol     0.5254    0.5254    0.5254       670
    high_vol     0.5254    0.5254    0.5254       670

    accuracy                         0.5254      1340
   macro avg     0.5254    0.5254    0.5254      1340
weighted avg     0.5254    0.5254    0.5254      1340


--------------------------------------------------------------------------------
[VOL] window=60min, bins=3


FinBERT zero-shot (vol, bins=3):   0%|          | 0/35 [00:00<?, ?it/s]

Accuracy = 0.3320
              precision    recall  f1-score   support

     low_vol     0.3347    0.3347    0.3347       744
     mid_vol     0.3226    0.3226    0.3226       744
    high_vol     0.3387    0.3387    0.3387       744

    accuracy                         0.3320      2232
   macro avg     0.3320    0.3320    0.3320      2232
weighted avg     0.3320    0.3320    0.3320      2232


--------------------------------------------------------------------------------
[VOL] window=60min, bins=7


FinBERT zero-shot (vol, bins=7):   0%|          | 0/35 [00:00<?, ?it/s]

Accuracy = 0.1443
               precision    recall  f1-score   support

     very_low     0.1567    0.1567    0.1567       319
          low     0.1254    0.1254    0.1254       319
 slightly_low     0.1536    0.1536    0.1536       319
       medium     0.1478    0.1473    0.1476       319
slightly_high     0.1505    0.1505    0.1505       319
         high     0.1473    0.1478    0.1476       318
    very_high     0.1285    0.1285    0.1285       319

     accuracy                         0.1443      2232
    macro avg     0.1443    0.1443    0.1443      2232
 weighted avg     0.1443    0.1443    0.1443      2232


=== Summary: zero-shot FinBERT baselines (log-return + vol) ===
logret | h= 1 min, bins=2: n_test=1311, acc=0.5233
logret | h= 1 min, bins=3: n_test=2184, acc=0.3297
logret | h= 1 min, bins=7: n_test=2184, acc=0.1387
logret | h= 5 min, bins=2: n_test=1311, acc=0.4867
logret | h= 5 min, bins=3: n_test=2185, acc=0.3396
logret | h= 5 min, bins=7: n_test=2185, acc=0.1318
log

In [ ]:
# ======================================================
# Case study: 30min, 2-class IV-normalized volatility + LoRA predictions
#  - 复用 ret_df, iv_long, news_filtered
#  - 只做 window=30, bins=2
#  - 生成 news_vol_30min_lora_preds.parquet
# ======================================================

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm
import torch

# 如果没定义 BASE_DIR，可以补一行：
try:
    BASE_DIR
except NameError:
    BASE_DIR = "/content/drive/MyDrive/COMS Shared Data"

WINDOW = 30            # 30min 窗口
NUM_BINS = 2           # 2-class: low_vol / high_vol
vol_col = f"vol_future_{WINDOW}m"

# ---------------- 1）从 ret_df + iv_long + news_filtered 拼 30min vol_norm ----------------

# ret_df: 在你 vol 实验的 cell 里已经从 ret_df_1min.parquet 读过了
# 如果你是单独跑这个 cell，可以取消下面这块注释来重新读
# RET_1MIN_PATH = os.path.join(BASE_DIR, "ret_df_1min.parquet")
# ret_df = pd.read_parquet(RET_1MIN_PATH)
# ret_df["time"] = pd.to_datetime(ret_df["time"])
# ret_df = ret_df.sort_values(["ticker", "time"]).reset_index(drop=True)
# ret_df["date"] = ret_df["time"].dt.date
# ret_df["log_ret"] = np.log1p(ret_df["ret_1m"])

# news_filtered: 在前面的 cell 是从 news_sorted.parquet 过来的
# 如果内存里没有，可以这样重建：
# news_sorted = pd.read_parquet(os.path.join(BASE_DIR, "news_sorted.parquet"))
# news_filtered = news_sorted[news_sorted["quality_score_model"] > 0.05].copy()

# iv_long: 在 vol 实验 cell 里是从 ATM IV 30.xlsx 算出来的
# 如果你单独跑，可以把那段 "iv_raw -> iv_long" 的代码搬过来

print("Rebuilding vol_norm for window=30min, bins=2 ...")

df = ret_df.copy()
df["time"] = pd.to_datetime(df["time"])
df["date"] = df["time"].dt.date
df["log_ret"] = np.log1p(df["ret_5m"])

g = df.groupby(["ticker", "date"], observed=True)

# realized vol of minute log returns, 30min window
df["vol_tmp"] = g["log_ret"].rolling(WINDOW).std().reset_index(level=[0, 1], drop=True)
df[vol_col] = g["vol_tmp"].shift(-WINDOW)
df = df.drop(columns=["vol_tmp"])

vol_df = df.dropna(subset=[vol_col])[["ticker", "time", vol_col]].copy()

# 建一个 {ticker -> DatetimeIndex} 方便 anchor
available = vol_df[["ticker", "time"]].drop_duplicates().sort_values(["ticker", "time"])
ticker2times = {}
for t, sub in available.groupby("ticker", observed=True):
    ticker2times[t] = pd.DatetimeIndex(sub["time"])

def compute_anchor(row):
    t = row["ticker"]
    ts = row["news_time"]
    tf = row["time_floor"]
    times = ticker2times.get(t)
    if times is None or len(times) == 0:
        return pd.NaT
    # 如果刚好对齐到某分钟
    if tf in times:
        return tf
    # 否则找第一个 >= ts 的分钟
    idx = times.searchsorted(ts)
    if idx >= len(times):
        return pd.NaT
    return times[idx]

news_df = news_filtered.copy()
news_df["news_time"] = pd.to_datetime(news_df["time"]).dt.tz_localize(None)
news_df["time_floor"] = news_df["news_time"].dt.floor("min")
news_df["anchor_time"] = news_df.apply(compute_anchor, axis=1)

anchors = news_df[news_df["anchor_time"].notna()].copy()
merged_df = anchors.merge(
    vol_df,
    left_on=["ticker", "anchor_time"],
    right_on=["ticker", "time"],
    how="inner",
)

merged_df = merged_df.dropna(subset=[vol_col]).sort_values(["anchor_time", "ticker"]).reset_index(drop=True)

# 合并 iv_prev
merged_df["anchor_date"] = pd.to_datetime(merged_df["anchor_time"]).dt.normalize()
merged_df = merged_df.merge(
    iv_long[["DATES", "ticker", "iv_prev"]],
    left_on=["anchor_date", "ticker"],
    right_on=["DATES", "ticker"],
    how="left",
).drop(columns=["DATES"])

# 归一化 vol
merged_df["vol_norm"] = merged_df[vol_col] / merged_df["iv_prev"]
merged_df = merged_df.dropna(subset=["vol_norm"])

print(f"Samples after vol + IV merge: {len(merged_df):,}")

# ---------------- 2）2-class 分箱（30% / 70% tails） + 文本清洗 ----------------

df_bin = merged_df.copy()

q_low = df_bin["vol_norm"].quantile(0.3)
q_high = df_bin["vol_norm"].quantile(0.7)
labels = np.full(len(df_bin), -1, dtype=int)
labels[df_bin["vol_norm"] <= q_low] = 0
labels[df_bin["vol_norm"] >= q_high] = 1

df_bin["label"] = labels
df_bin = df_bin[df_bin["label"] != -1].copy()
print(f"After tail filter: {len(df_bin):,} samples, unique labels={df_bin['label'].nunique()}")

df_bin["summary_clean"] = df_bin["summary"].apply(clean_text)
df_bin["text"] = df_bin.apply(build_input, axis=1)

label_names = ["low_vol", "high_vol"]

# ---------------- 3）train/val/test split ----------------

train_df, temp_df = train_test_split(
    df_bin,
    test_size=0.2,
    random_state=42,
    stratify=df_bin["label"],
)
val_df, test_df = train_test_split(
    temp_df,
    test_size=0.5,
    random_state=42,
    stratify=temp_df["label"],
)

print(
    f"Train={len(train_df):,}, Val={len(val_df):,}, Test={len(test_df):,}"
)

# ---------------- 4）搭 DataLoader，训练一个 LoRA 模型 ----------------

train_ds = SimpleTextDataset(train_df, text_col="text", label_col="label")
val_ds   = SimpleTextDataset(val_df,   text_col="text", label_col="label")

train_loader = DataLoader(
    train_ds,
    batch_size=BATCH_SIZE_TRAIN,
    shuffle=True,
    collate_fn=vol_collate_fn,
)
val_loader = DataLoader(
    val_ds,
    batch_size=BATCH_SIZE_TEST,
    shuffle=False,
    collate_fn=vol_collate_fn,
)

model = FinBertClassifierLoRA(encoder_name, num_labels=NUM_BINS).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

def eval_accuracy(model, data_loader):
    model.eval()
    all_labels, all_preds = [], []
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels_np = batch["labels"].cpu().numpy()

            logits = model(input_ids=input_ids, attention_mask=attention_mask)["logits"]
            preds_np = logits.argmax(dim=-1).cpu().numpy()

            all_labels.append(labels_np)
            all_preds.append(preds_np)
    all_labels = np.concatenate(all_labels)
    all_preds = np.concatenate(all_preds)
    acc = (all_labels == all_preds).mean()
    return acc

best_state = None
best_val_acc = -1.0

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0.0
    for batch in tqdm(train_loader, desc=f"LoRA train epoch {epoch+1} (30m, 2cls)", leave=False):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels_tensor = batch["labels"].to(device)

        optimizer.zero_grad()
        out = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels_tensor,
        )
        loss = out["loss"]
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / max(1, len(train_loader))
    val_acc = eval_accuracy(model, val_loader)
    print(f"[epoch {epoch+1}] train_loss={avg_loss:.4f}, val_acc={val_acc:.4f}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}

if best_state is not None:
    model.load_state_dict(best_state)

print(f"Best val_acc={best_val_acc:.4f}")

# ---------------- 5）在 test_df 上跑 LoRA，生成 parquet ----------------

model.eval()
test_texts = test_df["text"].tolist()
test_labels = test_df["label"].astype(int).to_numpy()

all_preds = []
for i in range(0, len(test_texts), BATCH_SIZE_TEST):
    batch_texts = test_texts[i : i + BATCH_SIZE_TEST]
    enc = tokenizer(
        batch_texts,
        padding=True,
        truncation=True,
        max_length=MAX_LEN,
        return_tensors="pt",
    )
    enc = {k: v.to(device) for k, v in enc.items()}
    with torch.no_grad():
        logits = model(input_ids=enc["input_ids"], attention_mask=enc["attention_mask"])["logits"]
    preds = logits.argmax(dim=-1).cpu().numpy()
    all_preds.append(preds)

all_preds = np.concatenate(all_preds)[: len(test_df)]
test_acc = (all_preds == test_labels).mean()
print(f"Test accuracy (30min, 2-class, LoRA) = {test_acc:.4f}")

test_df = test_df.copy()
test_df["pred_lora"] = all_preds

# 只保留 case study 需要的列
cols_keep = [
    "ticker",
    "anchor_time",
    "vol_norm",
    "label",
    "pred_lora",
    "summary",
    "text",
]
df_preds = test_df[cols_keep].copy()

out_path = os.path.join(BASE_DIR, "news_vol_30min_lora_preds.parquet")
df_preds.to_parquet(out_path, index=False)
print("Saved case-study parquet to:", out_path)

# ---------------- 6）从 parquet 里挑 LoRA 预测对的 high/low vol 新闻 ----------------

# 你也可以以后单独 read_parquet，这里顺手直接用 df_preds
df_correct = df_preds[df_preds["label"] == df_preds["pred_lora"]].copy()

df_high = df_correct.sort_values("vol_norm", ascending=False).head(5)
df_low  = df_correct.sort_values("vol_norm", ascending=True).head(5)

print("\n=== Top 5 high-vol cases (LoRA correct) ===")
for _, row in df_high.iterrows():
    print("-" * 80)
    print(f"Ticker: {row['ticker']}   Time: {row['anchor_time']}   vol_norm={row['vol_norm']:.4f}")
    print(f"True label: {row['label']} (high_vol)")
    print("Summary:")
    print(str(row["summary"]).replace("\n", " "))
    print()

print("\n=== Top 5 low-vol cases (LoRA correct) ===")
for _, row in df_low.iterrows():
    print("-" * 80)
    print(f"Ticker: {row['ticker']}   Time: {row['anchor_time']}   vol_norm={row['vol_norm']:.4f}")
    print(f"True label: {row['label']} (low_vol)")
    print("Summary:")
    print(str(row["summary"]).replace("\n", " "))
    print()


Rebuilding vol_norm for window=30min, bins=2 ...
Samples after vol + IV merge: 22,315
After tail filter: 13,405 samples, unique labels=2
Train=10,724, Val=1,340, Test=1,341


pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

LoRA train epoch 1 (30m, 2cls):   0%|          | 0/671 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

[epoch 1] train_loss=0.6965, val_acc=0.5381


...

[epoch 3] train_loss=0.6619, val_acc=0.5694


LoRA train epoch 4 (30m, 2cls):   0%|          | 0/671 [00:00<?, ?it/s]

[epoch 4] train_loss=0.6487, val_acc=0.5851


LoRA train epoch 5 (30m, 2cls):   0%|          | 0/671 [00:00<?, ?it/s]

[epoch 5] train_loss=0.6286, val_acc=0.5769
Best val_acc=0.5851
Test accuracy (30min, 2-class, LoRA) = 0.5712
Saved case-study parquet to: /content/drive/MyDrive/COMS Shared Data/news_vol_30min_lora_preds.parquet

=== Top 5 high-vol cases (LoRA correct) ===
--------------------------------------------------------------------------------
Ticker: PFE   Time: 2024-11-15 09:30:00   vol_norm=0.0287
True label: 1 (high_vol)
Summary:
Shares of Moderna, Novavax, Pfizer, BioNTech and GSK closed lower on Thursday.

--------------------------------------------------------------------------------
Ticker: AMZN   Time: 2022-05-02 09:30:00   vol_norm=0.0234
True label: 1 (high_vol)
Summary:
On another difficult Friday afternoon for the stock market, the broad U.S. indexes ended with significant declines --- with tech stocks standing out.

--------------------------------------------------------------------------------
Ticker: LLY   Time: 2024-02-20 09:30:00   vol_norm=0.0233
True label: 1 (high_vol)


In [ ]:
ret_df

,ticker,time,ret_5m
5060793,AAPL,2022-03-01 09:30:00,0.006618
5060792,AAPL,2022-03-01 09:31:00,0.004783
5060791,AAPL,2022-03-01 09:32:00,0.003986
5060790,AAPL,2022-03-01 09:33:00,0.003803
5060789,AAPL,2022-03-01 09:34:00,0.005109
...,...,...,...
16932240,XOM,2025-11-12 15:51:00,0.000042
16932239,XOM,2025-11-12 15:52:00,-0.000169
16932238,XOM,2025-11-12 15:53:00,-0.000635
16932237,XOM,2025-11-12 15:54:00,-0.000550
